## Data Pre-Processing

#### Import Functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.neighbors import BallTree
from sklearn.cluster import DBSCAN
from collections import Counter
from itertools import groupby
from itertools import chain
import pytz
import geopandas as gpd
from shapely.geometry import MultiPoint
from geopy.distance import great_circle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn.neighbors import NearestNeighbors
from kneed import KneeLocator
import math
from rdp_modified import rdp_mod
import time
import networkx as nx
import datetime
import matplotlib.colors as mcolors
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

# folder_loc = r"C:\Users\nh2u20\OneDrive - University of Southampton\Uni\3rd Year\Individual Project"
folder_loc = r"C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project"

#### Initial Data Cleaning

In [ ]:
#%% DATA PRE-PROCESSING DATA

cols = ['IMO', 'MMSI', 'latitude', 'longitude', 'sog', 'datetime']
df_2019 = pd.read_csv(folder_loc + r"\bulk_carrier_data\AIS_full_year_dates_parsed.csv", usecols=cols)
df_2019 = df_2019.rename(columns={'datetime': 'last_seen', 'sog': 'speed'})

cols = ['IMO', 'last_seen', 'latitude', 'longitude', 'speed']
df_2022 = pd.read_csv(folder_loc + r"\seanet\seanetships.csv", usecols=cols)
df_2023 = pd.read_csv(folder_loc + r"\seanet\olden_new_2023.csv", usecols=cols)

def clean_dataframes(dfs):
    dfs_out = []
    for df in dfs:
        df = df.rename(columns={'latitude': 'lat', 'longitude': 'lon', 'last_seen': 'datetime', 'speed': 'sog'})
        df['datetime'] =  pd.to_datetime(df['datetime'], dayfirst=True)
        df = df.drop_duplicates(subset=['IMO', 'datetime'])
        df = df.sort_values(by=['IMO', 'datetime'], ascending=[True, True])
        df = df[['IMO', 'lat', 'lon', 'sog', 'datetime']] # reorder cols
        df['sog'] = df['sog'].fillna(0)    
        df['IMO'] = df['IMO'].astype(str)
        dfs_out.append(df)
    return dfs_out

dfs = clean_dataframes([df_2019, df_2022, df_2023])

df = pd.concat(dfs, axis=0)
df = df.reset_index(drop=True)

#### Plotting Unprocessed Data

In [ ]:
# %matplotlib inline
# plt.figure(figsize=(8,8), dpi=400)
# ax = plt.axes(projection=ccrs.PlateCarree())
# # ax.add_feature(cfeature.LAND, facecolor="lightgrey")
# gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, 45), ylocs=range(-90, 90, 45))
# # ax.coastlines()
# ax.scatter(df.lon, df.lat, c='k', edgecolor='None', marker = "x", alpha=0.7, s=0.2, linewidth=0.02)
# gl.top_labels = False
# gl.right_labels = False
# gl.xlabel_style = {'size': 8, 'color': 'k'}
# gl.ylabel_style = {'size': 8, 'color': 'k'}
# gl.xpadding = 15 
# gl.ypadding = 10 

In [ ]:
# a =  pd.read_csv(folder_loc + r"\bulk_carrier_data\AIS_full_year_dates_parsed.csv")
# b = pd.read_csv(folder_loc + r"\seanet\seanetships.csv")
# c = pd.read_csv(folder_loc + r"\seanet\olden_new_2023.csv")

# nan_count = b["draft"].isna().sum()
# nan_percentage = nan_count / len(b) * 100
# print(f"{nan_percentage:.2f}%")


#### Plotting Unprocessed Data

In [ ]:
# %matplotlib inline
# plt.figure(figsize=(8,8), dpi=400)
# ax = plt.axes(projection=ccrs.PlateCarree())
# # ax.add_feature(cfeature.LAND, facecolor="lightgrey")
# gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, 45), ylocs=range(-90, 90, 45))
# # ax.coastlines()
# ax.scatter(df.lon, df.lat, c='k', edgecolor='None', marker = "x", alpha=0.7, s=0.2, linewidth=0.02)
# gl.top_labels = False
# gl.right_labels = False
# gl.xlabel_style = {'size': 8, 'color': 'k'}
# gl.ylabel_style = {'size': 8, 'color': 'k'}
# gl.xpadding = 15 
# gl.ypadding = 10 

In [ ]:
# a =  pd.read_csv(folder_loc + r"\bulk_carrier_data\AIS_full_year_dates_parsed.csv")
# b = pd.read_csv(folder_loc + r"\seanet\seanetships.csv")
# c = pd.read_csv(folder_loc + r"\seanet\olden_new_2023.csv")

# nan_count = b["draft"].isna().sum()
# nan_percentage = nan_count / len(b) * 100
# print(f"{nan_percentage:.2f}%")


#### Filter out Erroneous Speeds

In [ ]:
df = pd.read_pickle('df1.pkl')
df.loc[df['sog'] == 102.3, 'sog'] = np.nan # replace anomalous sog values with nan
df['sog'] = df['sog'].interpolate(method='pad')
df = df[df['sog'] < 20]
# df1.sog.plot.kde()

##### Other Useful Functions

In [ ]:
def haversine_np(lon1, lat1, lon2, lat2, in_deg=True):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    if in_deg:
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])


    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371.0088 * c
    return km


In [ ]:
def plot_wrapping(df, geo_map=True):
    """
    prevent line from plotting across map when position goes from +180 to -180
    """
    
    # Convert longitudes greater than 180 to negative values
    df.loc[df['lon'] > 180, 'lon'] -= 360
    
    # Convert longitudes less than -180 to positive values
    df.loc[df['lon'] < -180, 'lon'] += 360
    
    x, y = df['lon'], df['lat']
    
    # split the track into segments that dont cross the dateline
    xdiff = np.diff(x)
    wrapidx = np.where(np.abs(xdiff) > 180)[0] + 1
    segments = np.split(df, wrapidx)   
    
    plt.figure(dpi=300)
    
    if geo_map:
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.coastlines(linewidth=0.35)
        # ax.add_feature(cfeature.LAND)
        # ax.add_feature(cfeature.OCEAN)
        # ax.add_feature(cfeature.BORDERS, linestyle='-', alpha=.5)
    
    
    # Plot each segment separately
    for i, seg in enumerate(segments):
        x, y = seg['lon'], seg['lat']
        plt.plot(x, y, linewidth=0.1, c="k", transform=ccrs.Geodetic())
        plt.xlabel("longitude")
        plt.ylabel("latitude")
        

#### Drop Erroneous Positions

In [ ]:
#%% DROP ERRONEOUS POSITIONS

def calc_distances(df):
    df['lat1'] = df['lat'].shift(1)
    df['lon1'] = df['lon'].shift(1)
    df['dist'] = haversine_np(df['lon'], df['lat'], df['lon1'], df['lat1'])
    df['time_diff'] = df['datetime'].diff()
    df['leg_speed'] = df['dist']*1000 / df['time_diff'].dt.total_seconds() # in m/s
    df['leg_speed_kts'] = df['leg_speed']*1.9438452
    # df['speed_diff'] = df['leg_speed_kts'] - df['sog']

    df = df.drop(columns=['lat1', 'lon1'])

    return df


def drop_err_positions(df, cutoff_speed=20, plots=False):

    df2 = pd.DataFrame()
    for ship in df.groupby('IMO'):
        df_temp = ship[1]
        df_temp = calc_distances(df_temp)
        i = 0
        df_ori = df_temp.copy()
        max_sp = df_temp['leg_speed_kts'].max()
        
        while len(df_temp.loc[(df_temp['leg_speed_kts'] > cutoff_speed)]) != 0:
            i = i + 1
            df_temp = df_temp[df_temp['leg_speed_kts'] < cutoff_speed]
            df_temp = calc_distances(df_temp)
        
        df2 = df2.append(df_temp)
        
        if plots:
            if i > 0:
                plot_wrapping(df_ori, geo_map=True)
                plt.scatter(df_ori.lon, df_ori.lat, s=0.8, c=df_ori.datetime, alpha=0.7)
                plt.title(f'{i} erroneous position records {max_sp}')
            print(f'dropped {i} records')


    return df2

df2 = drop_err_positions(df, plots=False)


In [ ]:
# # Coastline data to detect ports
# coastlines = pd.read_csv(folder_loc + r"\other\coasts.csv")
# coastlines = coastlines.rename(columns={'latitude': 'lat', 'longitude': 'lon'})

In [ ]:
# import geopandas as gpd
# import os

# dir_path = r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\gshhg-shp-2.3.7\WDBII_shp\h'

# # Get a list of all shapefiles in the directory
# # shp_files = [f for f in os.listdir(dir_path) if f.endswith('.shp')]
# shp_files = [f for f in os.listdir(dir_path) if f.endswith('.shp') and 'river' in f]
# # Loop through each shapefile and extract the linestring data
# point_data = []
# for shp_file in shp_files:
#     gdf = gpd.read_file(os.path.join(dir_path, shp_file))
#     # gdf.plot()
#     # Extract the point data from the linestrings
#     for row in gdf.itertuples():
#         for point in row.geometry.coords:
#             point_data.append((point[0], point[1]))
    
# # Create a dataframe from the point data
# dfaa = pd.DataFrame(point_data, columns=['longitude', 'latitude'])
# print(len(dfaa))
# # Write the dataframe to a CSV file
# dfaa.to_csv(r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\rivers.csv', index=False)

In [ ]:
# import geopandas as gpd
# import os

# dir_path = r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\gshhg-shp-2.3.7\GSHHS_shp\h'

# # Get a list of all shapefiles in the directory
# shp_files = [f for f in os.listdir(dir_path) if f.endswith('.shp')]

# # Loop through each shapefile and extract the point data
# point_data = []
# for shp_file in shp_files:
#     gdf = gpd.read_file(os.path.join(dir_path, shp_file))
#     # Extract the point data from the polygons
#     for row in gdf.itertuples():
#         polygon = row.geometry
#         for point in polygon.exterior.coords:
#             point_data.append((point[0], point[1]))

# # Create a dataframe from the point data
# dfz = pd.DataFrame(point_data, columns=['longitude', 'latitude'])
# print(len(dfz))
# # Write the dataframe to a CSV file
# dfz.to_csv(r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\shorelines.csv', index=False)



In [ ]:
%matplotlib inline
max_time_dict = {}
print(df2['datetime'].dtype)
for i, (imo, df_temp) in enumerate(df2.groupby('IMO')):         
    max_time = df_temp['time_diff'].dt.days.max()
    max_time_dict[imo] = max_time

plt.figure(figsize=[10, 4], dpi=400)
x_labels = list(map(str, max_time_dict.keys())) 
y_values = list(max_time_dict.values())

colors = ['grey' if y >= 10 else 'lightgrey' for y in y_values]

plt.bar(x_labels, y_values, align='edge', width=0.8, color=colors)
plt.xlabel('Ship IMO')
plt.xlim(x_labels[0], x_labels[-1])
plt.tick_params(axis='x', labelbottom=False)
plt.ylabel('Maximum Time Gap (days)')
plt.yscale('log')
plt.axhline(y=10, color='black', linestyle='--')
plt.show()

In [ ]:
# for i, (imo, df_temp) in enumerate(df2.groupby('IMO')):
#     plt.figure()
#     ax = plt.axes(projection=ccrs.PlateCarree())
#     ax.coastlines(linewidth=0.35)
    
#     plt.scatter(df_temp.lon, df_temp.lat, s=0.1, linewidth=0.5)

In [ ]:
#%% SPLIT SHIPS WITH TIMEGAP INTO MULTIPLE SHIPS

def split_ships(df, cutoff_time=10):
    """splits a ship into multiple ships if time diff between succesive records > cutoff_time (in days)"""

    df2 = pd.DataFrame()

    for i in df.groupby('IMO'):
        ship = i[1]
        cut_time = pd.Timedelta(days=cutoff_time)
        
        ship['mask'] = ship['time_diff'] > cut_time
        ship['mask'] = ship['mask'].fillna(method='bfill')
        ship['mask']= ship['mask'].astype(int)

        if ship['mask'].cumsum().iloc[-1] > 0:
            ship['IMO'] = ship['IMO'] + '_' + ship['mask'].cumsum().astype(str)

        for j in ship.groupby('IMO'):
            sub_ship = j[1]
            sub_ship = calc_distances(sub_ship)
            sub_ship = sub_ship.drop(columns=['mask'])
            df2 = df2.append(sub_ship)

    return df2

df = split_ships(df2)

## Ports Database

#### Ships Close to Coastline

In [ ]:
# # Coastline data to detect ports
# coastlines = pd.read_csv(folder_loc + r"\other\coasts.csv")
# coastlines = coastlines.rename(columns={'latitude': 'lat', 'longitude': 'lon'})

In [ ]:
# import geopandas as gpd
# import os

# dir_path = r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\gshhg-shp-2.3.7\WDBII_shp\h'

# # Get a list of all shapefiles in the directory
# # shp_files = [f for f in os.listdir(dir_path) if f.endswith('.shp')]
# shp_files = [f for f in os.listdir(dir_path) if f.endswith('.shp') and 'river' in f]
# # Loop through each shapefile and extract the linestring data
# point_data = []
# for shp_file in shp_files:
#     gdf = gpd.read_file(os.path.join(dir_path, shp_file))
#     # gdf.plot()
#     # Extract the point data from the linestrings
#     for row in gdf.itertuples():
#         for point in row.geometry.coords:
#             point_data.append((point[0], point[1]))
    
# # Create a dataframe from the point data
# dfaa = pd.DataFrame(point_data, columns=['longitude', 'latitude'])
# print(len(dfaa))
# # Write the dataframe to a CSV file
# dfaa.to_csv(r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\rivers.csv', index=False)

In [ ]:
# import geopandas as gpd
# import os

# dir_path = r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\gshhg-shp-2.3.7\GSHHS_shp\h'

# # Get a list of all shapefiles in the directory
# shp_files = [f for f in os.listdir(dir_path) if f.endswith('.shp')]

# # Loop through each shapefile and extract the point data
# point_data = []
# for shp_file in shp_files:
#     gdf = gpd.read_file(os.path.join(dir_path, shp_file))
#     # Extract the point data from the polygons
#     for row in gdf.itertuples():
#         polygon = row.geometry
#         for point in polygon.exterior.coords:
#             point_data.append((point[0], point[1]))

# # Create a dataframe from the point data
# dfz = pd.DataFrame(point_data, columns=['longitude', 'latitude'])
# print(len(dfz))
# # Write the dataframe to a CSV file
# dfz.to_csv(r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\shorelines.csv', index=False)



In [ ]:
# %matplotlib inline
rivers = pd.read_csv(folder_loc + r"\other\rivers.csv")
rivers = rivers.rename(columns={'latitude': 'lat', 'longitude': 'lon'})

shorelines = pd.read_csv(folder_loc + r"\other\shorelines.csv")
shorelines = shorelines.rename(columns={'latitude': 'lat', 'longitude': 'lon'})


# plt.figure(figsize=(8,8), dpi=400)
# ax = plt.axes(projection=ccrs.PlateCarree())

# plt.scatter(rivers.lon, rivers.lat, s=0.005, label="rivers", c="#008CFF", linewidths=0.1, alpha=0.5)
# plt.scatter(shorelines.lon, shorelines.lat, s=0.01, label="shorelines", c="k", linewidths=0.1)
# ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())

# gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.5, xlocs=range(-180, 181, 90), ylocs=range(-180, 91, 45))
# gl.top_labels = True
# gl.right_labels = True
# gl.xlabel_style = {'size': 8, 'color': 'k'}
# gl.ylabel_style = {'size': 8, 'color': 'k'}
# # plt.legend()
# gl.xpadding = 15 
# gl.ypadding = 10 

coasts_rivers = pd.concat([shorelines, rivers], ignore_index=True)



Source of first two functions: https://autogis-site.readthedocs.io/en/2020_/notebooks/L3/06_nearest-neighbor-faster.html

In [ ]:
#%% SHIPS CLOSE TO PORTS

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine') 
    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    closest = indices[0] # 0 for closest, 1 would be second closest
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)

def nearest_neighbor(left_gdf, right_gdf, return_dist=True):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.
    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """
    
    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name
    
    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in km)
    
    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]
    
    # Ensure that the index corresponds the one in left_gdf
    # closest_points = closest_points.reset_index(drop=True)
    closest_points = closest_points.set_index(left_gdf.index)
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371.0088  # km
        closest_points['distance'] = dist * earth_radius
        
    return closest_points


def create_geodf(df):
    """create a geodataframe from a dataframe"""
    geom = gpd.points_from_xy(df.lon, df.lat, crs="EPSG:4326")
    gdf = gpd.GeoDataFrame(df, geometry=geom)    
    return gdf

def get_nn_df(df_main, df_comp, keep_id=False, id='id', id_col=False):
    """returns df with additional columns:
        - the closest data point to another dataframe, df_comp
        - its respective distance"""

    gdf_main = create_geodf(df_main)
    gdf_comp =  create_geodf(df_comp)

    # FIND CLOSEST PORT FOR EACH AIS SIGNAL
    closest_geom = nearest_neighbor(gdf_main, gdf_comp, return_dist=True) 
    closest_geom = closest_geom.set_index(df_main.index)
    if id_col:
        closest_geom = closest_geom.rename(columns={'geometry': 'closest_geom', id: f'closest_{id}'})
    else:
        closest_geom = closest_geom.rename(columns={'geometry': 'closest_geom'})

    if keep_id:
        try:
            closest_geom = closest_geom[[f'closest_{id}','closest_geom', 'distance']]
        except:
            closest_geom = closest_geom[[id, 'closest_geom', 'distance']]
    else:
        closest_geom = closest_geom[['closest_geom', 'distance']]
    # joined dataframes
    df_closest_geom = gdf_main.join(closest_geom).copy()
    df_closest_geom = df_closest_geom.set_index(df_main.index)

    return df_closest_geom


def get_df_within_dist(df_main, df_comp, within_dist, keep_id=False, id='id', id_col=False):
    """Returns a dataframe of the rows in df_main within_dist (km) of a second dataframe df_comp"""

    df_closest_geom = get_nn_df(df_main, df_comp, keep_id, id, id_col)
    # only retain ships within certain km of a port
    gdf_df_near_geom = df_closest_geom.loc[df_closest_geom['distance'] < within_dist] # distance in km
  
    # gdf_df_near_geom = gdf_df_near_geom.drop(columns=["geometry", "closest_geom"])
    gdf_df_near_geom = gdf_df_near_geom.drop(columns=["geometry", "closest_geom", "distance"])

    return gdf_df_near_geom

df_slow = df[df.sog < 1] # slow ships (below 1kts)
ships_in_port = get_df_within_dist(df_slow, coasts_rivers, within_dist=10)


In [ ]:
# from shapely.geometry import LineString

# # Create a link (LineString) between building and stop points

# ships_in_port['link'] = ships_in_port.apply(lambda row: LineString([row['geometry'], row['closest_geom']]), axis=1)

# # Set link as the active geometry
# ship_links = ships_in_port.copy()
# ship_links = ship_links.set_geometry('link')

# plt.figure()
# ax = plt.axes(projection=ccrs.PlateCarree())

# # Plot the connecting links between ships and ports and color them based on distance
# ship_links.plot(ax=ax, column='distance', cmap='Blues', scheme='quantiles', k=4, alpha=0.8, lw=1)
# ships_in_port.plot(ax=ax, color='blue', markersize=8, alpha=0.7)
# ax.scatter(coastlines.lon, coastlines.lat, c="gray", label="Coastline", s=8, linewidth=0.1)

# # coasts_rivers.plot(ax=ax, markersize=4, marker='o', color='red', alpha=0.9, zorder=3)

# # Zoom closer
# ax.set_xlim([-180, 180])
# ax.set_ylim([-90, 90])
# plt.title("Nearest Neighbours")
# # Set map background color to black, which helps with contrast
# # ax.set_facecolor('black')

##### Ships in Port Global

In [ ]:
%matplotlib inline
plt.figure(figsize=(6,6), dpi=400)
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.RIVERS, linewidth=0.4)
ax.add_feature(cfeature.BORDERS, linewidth=0.2, color="gray")
gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, 45), ylocs=range(-90, 90, 45))
# # ax.coastlines()
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 8, 'color': 'k'}
gl.ylabel_style = {'size': 8, 'color': 'k'}
gl.xpadding = 15 
gl.ypadding = 10 

plt.scatter(x=df.lon, y=df.lat, s=0.05, c="black", label="Ship Route", zorder=3, alpha=0.5, linewidth=0.01)
plt.scatter(x=df_slow.lon, y=df_slow.lat, s=1, c='tab:red', label="Stationary Ship", zorder=4, linewidth=0.1)
plt.scatter(x=ships_in_port.lon, y=ships_in_port.lat, s=1, c="limegreen", label="Ship in Port", zorder=5, linewidth=0.1)
plt.legend(loc='upper center',bbox_to_anchor=(0.7,0.1), markerscale=5, ncol=3, prop={'size': 6}, frameon=False)
plt.show()



##### Ships in Port Local

In [ ]:
#%% PLOTTING SHIPS IN PORT
%matplotlib inline
plt.figure(figsize=[4,4], dpi=400)

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([130, 140, -18, -7], crs=ccrs.PlateCarree())

ax.coastlines(linewidth=0.35)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
# ax.add_feature(cfeature.BORDERS, linestyle='-', alpha=.5, linewidth=0.5)
ax.add_feature(cfeature.LAKES, alpha=0.95)

# plt.scatter(x=coastlines.lon, y=coastlines.lat, s=0.1, c="k", label="Coastline", zorder=5)
plt.scatter(x=df.lon, y=df.lat, s=2, c="black", label="Ship route", zorder=3, linewidths=0.1, alpha=0.5)
plt.scatter(x=df_slow.lon, y=df_slow.lat, s=30, c='tab:red', label="Stationary ship", zorder=4, linewidths=0.1)
plt.scatter(x=ships_in_port.lon, y=ships_in_port.lat, s=30, c="orange", label="Ship near coast", zorder=5, linewidths=0.1)

plt.legend(loc='lower left', markerscale=2, prop={'size': 10}, frameon=False, handletextpad=0.01, labelspacing=0.6, bbox_to_anchor=(-0.05, 0))
# plt.legend(loc='upper center',bbox_to_anchor=(0.5,0.08), markerscale=2, ncol=3, prop={'size': 6}, frameon=False)
gl = ax.gridlines(draw_labels=True, alpha=0.3, color="k", linestyle='--', linewidth=0.8, xlocs=range(130, 150, 5), ylocs=range(-20, -7, 5))
gl.xpadding = 20 
gl.ypadding = 20 
gl.top_labels = False
gl.right_labels = False

plt.show()

In [ ]:
#%% PLOTTING SHIPS IN PORT
%matplotlib inline
plt.figure(figsize=[4, 4], dpi=400)

ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([48, 58, 22, 28], crs=ccrs.PlateCarree())

ax.coastlines(linewidth=0.35)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAKES, alpha=0.95)

plt.scatter(x=df.lon, y=df.lat, s=2, c="black", label="Ship route", zorder=3, linewidths=0.1, alpha=0.5)
plt.scatter(x=df_slow.lon, y=df_slow.lat, s=6, c='tab:red', label="Stationary ship", zorder=4, linewidths=0.1)
plt.scatter(x=ships_in_port.lon, y=ships_in_port.lat, s=20, c="orange", label="Ship near coast", zorder=5, linewidths=0.1)

plt.legend(loc='lower left', markerscale=2, prop={'size': 10}, frameon=False, handletextpad=0.01, labelspacing=0.4, bbox_to_anchor=(-0.05, 0))
gl = ax.gridlines(draw_labels=True, alpha=0.3, color="k", linestyle='--', linewidth=0.8, xlocs=range(48, 58, 4), ylocs=range(22, 28, 2))
gl.xpadding = 20 
gl.ypadding = 20 
plt.show()

#### DBSCAN Clustering Ships near coast to form Ports

In [ ]:
def extract_coords(df):
    """gets the latitude and longitude data from a dataframe and returns it as a numpy array"""
    df_coords = pd.DataFrame()
    df_coords["lat"] = df.lat
    df_coords["lon"] = df.lon
    np_coords = df_coords.values

    return np_coords

port_coords = extract_coords(ships_in_port)

In [ ]:
# neighbors = NearestNeighbors(n_neighbors=2)
# neighbors_fit = neighbors.fit(port_coords)
# distances, indices = neighbors_fit.kneighbors(port_coords)

# distances = np.sort(distances, axis=0)
# distances = distances[:,1]


# i = np.arange(len(distances))
# knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')

# fig = plt.figure(figsize=(5, 5))
# knee.plot_knee()
# plt.xlabel("Points")
# plt.ylabel("Distance")

# opt_eps = distances[knee.knee]
# print(opt_eps)

In [ ]:
# data = gpd.read_file(r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\World_Port_Index\World_Port_Index.shp')
# import scipy
# from scipy.spatial import cKDTree
# Build kdtree
# coordinates = np.radians(data[['LATITUDE', 'LONGITUDE']])
# kdtree = cKDTree(coordinates)

# # Query kdtree for closest points
# distances_degrees, indexes = kdtree.query(coordinates, k=2)

# R = 6371.0088  # earth's radius in km
# distances_km = 2 * R * np.sin(distances_degrees[:, 1] / 2)

# new_df = data.assign(ClosestID=data['FID'][indexes[:, 1]].values, ClosestDist=distances_km)
# zz = new_df.ClosestDist.unique()
# pct = sum(zz > 5) / len(zz) * 100

# print(f"{100-pct:.2f}% less than 5km")

 adapted from https://stackoverflow.com/questions/67968952/how-to-calculate-distance-of-coordinates-and-categorical-dataset-with-dbscan-alg


In [ ]:
#%% CLUSTER SHIPS DETECTED IN PORTS
def get_centermost_point(cluster): 
    "calculates the centroid of a cluster"
    # https://gis.stackexchange.com/questions/379042/getting-the-center-point-of-a-cluster-in-pandas
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)


def DBSCAN_km(coords, km=10, min_points=2, labels=False):
    """returns a dataframe of cluster coordinates and cluster ids formed by DBSCAN"""
    db = DBSCAN(eps=km/6371.0088, min_samples=min_points, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.DataFrame([coords[cluster_labels == n] for n in range(num_clusters)], columns = ['value'])
    clusters['Length'] = clusters.value.apply(lambda x: len(x))
    clusters = clusters[clusters.Length > 0]
    clusters = clusters['value']

    centermost_points = clusters.map(get_centermost_point)
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({'lat':lats, 'lon':lons})
    
    coords_clustered = np.apply_along_axis(lambda row: coords[(coords[:,0]==row[0]) & (coords[:,1]==row[1])][0], axis=1, arr=rep_points)
    coords_clustered = pd.DataFrame(coords_clustered, columns=['lat', 'lon'])
    coords_clustered = coords_clustered.reset_index(drop=True) 
    coords_clustered.index += 1 # start index from 1
    coords_clustered['port_id'] = coords_clustered.index

    cmap = plt.cm.get_cmap('turbo', num_clusters)
    colors = cmap(cluster_labels)
    

    if labels:
        return coords_clustered, cluster_labels

    else:
        return coords_clustered

ports_clustered, ports_labels = DBSCAN_km(port_coords, km=20, min_points=2, labels=True)
ships_in_port_cluster = get_df_within_dist(ships_in_port, ports_clustered, within_dist=10, keep_id=True, id='port_id')


#### Plotting DBSCAN Port Clusters

In [ ]:
%matplotlib inline
plt.figure(figsize=(4,4), dpi=400)
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
ax.add_feature(cfeature.LAND, facecolor="lightgrey")
gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, 2), ylocs=range(-90, 90, 2))
# # ax.coastlines()
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 8, 'color': 'k'}
gl.ylabel_style = {'size': 8, 'color': 'k'}
gl.xpadding = 15 
gl.ypadding = 10 


region = "CH"
if region == "CH":
    # ax.set_extent([117, 136, 32, 44], crs=ccrs.PlateCarree())
    ax.set_extent([118, 122.1, 37, 40], crs=ccrs.PlateCarree())

elif region =="EU":
    ax.set_extent([-5, 20, 40, 70], crs=ccrs.PlateCarree())
elif region == "US":
    ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
elif region == "ME":
    ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
elif region == "AF":
    ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())


for label in np.unique(ports_labels):
    mask = (ports_labels == label)
    x = port_coords[:, 1][mask]
    y = port_coords[:, 0][mask]

    if label == -1: # ignore -1 labels (noise)
        plt.scatter(x, y, s=8, c="lightgrey")

    else:
        plt.scatter(x, y, s=20)

plt.scatter(ports_clustered.lon, ports_clustered.lat, c="k", s=20, marker="x", linewidth=1.5)


In [ ]:
%matplotlib inline
plt.figure(figsize=(6,6), dpi=400)
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines()
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.RIVERS, linewidth=0.4)
ax.add_feature(cfeature.BORDERS, linewidth=0.2, color="gray")
gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, 45), ylocs=range(-90, 90, 45))
# # ax.coastlines()
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 8, 'color': 'k'}
gl.ylabel_style = {'size': 8, 'color': 'k'}
gl.xpadding = 15 
gl.ypadding = 10 


rs_scatter = ax.scatter(ports_clustered['lon'], ports_clustered['lat'], c='k', s=0.5, zorder=1, transform=ccrs.PlateCarree())
plt.show()


## Ship Data

#### Dataframe for each Ship

In [ ]:
#%% Dataframe of Dataframes for Ship Data

def remove_duplicates(arr):
    result = np.array([next(g) for k, g in groupby(arr)])
    return result


def get_status(row):
    #returns S if a vessel is "moored/anchored/stopped" but not inside a port
    if row['dist'] < 0.1 and row['sog'] < 1 and pd.isna(row['port_id']):
        return 'A' # anchored
    else:
        return 'S' # sailing


ships_to_append = []
ports_to_append = []
df_ports = pd.DataFrame()


def add_common_index(df_to_edit, df_to_compare, col_name="port_id", main_col_name="port_id"):
    common_index = df_to_compare.index.intersection(df_to_edit.index)
    df_to_edit.loc[common_index, main_col_name] = df_to_compare.loc[common_index, col_name] # port_id ship is in (only populated if ship is in a port)

    return df_to_edit


# loop through df to determine where ships are in relation to ports
for i, (imo, df_temp) in enumerate(df.groupby('IMO')):
    total_dist = df_temp['dist'].cumsum().max()

    #------------------------Add Port ID if Ship is in Port------------------
    df_temp = add_common_index(df_temp, ships_in_port_cluster, col_name="port_id")

    #------------------ ASSIGN CLOSEST PORT TO ANCHORED SHIPS----------------
    df_temp['status'] = df_temp.apply(get_status, axis=1)
       
    # get nearest port to status
    df_anchor = df_temp[df_temp['status'] == 'A']

    if len(df_anchor) > 0:
        ships_moored = get_df_within_dist(df_anchor, ports_clustered, within_dist=80, keep_id=True, id="port_id", id_col=True)
        df_temp = add_common_index(df_temp, ships_moored, col_name="closest_port_id", main_col_name="port_id")
        
    #------------------ CREATE BETW_PORT COLUMN -----------------------------
    df_temp["port_id_fwd"] = df_temp["port_id"].ffill() # replace following nans with current port id
    df_temp["port_id_bwd"] = df_temp["port_id"].bfill() # replace previous nans with current port id
    df_temp["subtract1"] = df_temp["port_id_bwd"] - df_temp["port_id"] # returns 0 if ship is in port, nan if not in port
    df_temp["subtract1"]  = df_temp["subtract1"].replace(np.NaN, 1) # if not in port, replace with 1
    df_temp["subtract1"]  = df_temp["subtract1"].replace(0, np.NaN) # if in port, replace with 0
    df_temp["from_port"] = df_temp["port_id_fwd"] * df_temp["subtract1"]
    df_temp["to_port"] = df_temp["port_id_bwd"] * df_temp["subtract1"]
    df_temp["betw_port"] = (list(zip(df_temp["from_port"], df_temp["to_port"])))
    
    # replace nan values for port with tuple of port id it is in
    df_temp['betw_port'] = np.where(df_temp['betw_port'].apply(lambda x: pd.isna(x[0]) and pd.isna(x[1])),  # condition
                                    df_temp['port_id'].apply(lambda x: (x, x)),  # new value
                                    df_temp['betw_port'])  # existing value
    
    df_temp = df_temp.drop(columns=["subtract1", "port_id_fwd", "port_id_bwd", "from_port", "to_port"])
    
    ports_visited = remove_duplicates(df_temp.sort_values("datetime").dropna().port_id.values)  

    #-------------- DELETE FIRST AND LAST SEGMENTS IF NOT IN PORT--------------
    # find the indices of the first and last non-null port_id
    first_non_nan = df_temp['port_id'].first_valid_index()
    last_non_nan = df_temp['port_id'].last_valid_index()
    if first_non_nan != None:
        df_temp = df_temp.loc[first_non_nan:last_non_nan]    
        
        #-----------------------------APPENDING------------------------------------
        # df_temp = df_temp.drop(columns=["dist"])

        record = {'IMO': imo, 'data': df_temp, 'total_dist': total_dist, 'ports_visited': ports_visited}
        ships_to_append.append(record)
        
        ports_seq = [(ports_visited[i], ports_visited[i+1]) for i in range(len(ports_visited)-1)]
        port_record = {"sequence": ports_seq, "IMO": imo}
        ports_to_append.append(port_record)
        
        df_ports = df_ports.append(df_temp)
            
                     
df_ships = pd.DataFrame.from_records(ships_to_append, index='IMO')  
port_sequences = pd.DataFrame.from_records(ports_to_append)


#### Port to Port Sequences

In [ ]:
#%% PORT-PORT SEQUENCES

def sequence_to_counter(df):
    """Takes as an input a dataframe of an array of tuples (e.g. [(1, 2), (2, 3)...])
    and returns a dataframe with the number of times a tuple combination occurs
    and a counter for each element within the tuple
    """

    # list of all port-to-port combinations
    tuple_list = [tup for sublist in df['sequence'] for tup in sublist]
    
    # number of occurrences of each port-to-port combination
    combs_counter = Counter(tuple_list)
    
    item_counter = df['sequence'].tolist()
    item_counter = list(chain.from_iterable(item_counter))
    item_counter = [value for tup in item_counter for value in tup]
    item_counter = Counter(item_counter)
    
    # total number of times a ship has visited the port (includes times when a ship has visited it multiple times a year)
    item_counter = dict(item_counter.most_common())
    
    return combs_counter, item_counter


port_comb_counts, most_common_ports = sequence_to_counter(port_sequences)
ports_clustered['no_ships_visited'] = pd.Series(most_common_ports)

# port-to-port sequences stats
port_to_port = pd.DataFrame([port_comb_counts]).transpose()
port_to_port.rename(columns={0:'no_times'},inplace=True)

## Prepping Data for Graph

#### Plot RDP wrapping 

In [ ]:
def d_cross_track(latA, lonA, latB, lonB, latP, lonP):
    """Returns the shortest distance(km) between an arc AB 
       and a point in question P. 
    """

    latA, lonA, latB, lonB, latP, lonP = map(np.radians, [latA, lonA, latB, lonB, latP, lonP])

    R = 6371.0088  # Earth radius in km

    bngAB = bng(latA, lonA, latB, lonB)
    bngAC = bng(latA, lonA, latP, lonP)
    distAP = haversine_np(lonA, latA, lonP, latP, in_deg=False)

    diff = abs(bngAC - bngAB)
    if diff > math.pi:
        diff = 2 * math.pi - diff
    # Is relative bearing obtuse?
    if diff > (math.pi/2):
        d = distAP
    else:
        # Find the cross-track distance.
        d_temp = math.asin(math.sin(distAP/R) * math.sin(bngAC - bngAB)) * R

        # Is p4 beyond the arc?
        distAB = haversine_np(lonA, latA, lonB, latB, in_deg=False)
        distAI = math.acos(math.cos(distAP/R) / math.cos(d_temp/R)) * R # I is the point of intersection on great circle
        if distAI > distAB:
            d = haversine_np(lonB, latB, lonP,  latP, in_deg=False)
        else:
            d = abs(d_temp)

    return d


def bng(latA, lonA, latB, lonB):
    # Returns the bearing from one lat/lon point in radians to another
    b = math.atan2(math.sin(lonB - lonA) * math.cos(latB),
                   math.cos(latA) * math.sin(latB) - math.sin(latA) * math.cos(latB) * math.cos(lonB - lonA))
    return b


def rdp_idxs(input, epsilon, dist=d_cross_track):
    """
    Simplifies a given array of points.
    """
    dmax = 0.0
    index = -1

    for i in range(1, len(input)):
        d = dist(*input[0], *input[-1], *input[i])

        if d > dmax:
            index = i
            dmax = d

    if dmax > epsilon:
        idx_st1 = rdp_idxs(input[:index + 1], epsilon, dist) # sub track 1 index
        idx_st2 = rdp_idxs(input[index:], epsilon, dist)     # sub track 2 index
        
        # Add the index of the last point of r1 to r2, since they are neighbors
        idx_st2 = [x + index for x in idx_st2]

        return idx_st1[:-1] + idx_st2
    else:
        return [0, len(input) - 1]


In [ ]:
def plot_wrapping2(df):
    # Convert longitudes greater than 180 to negative values
    df.loc[df['lon'] > 180, 'lon'] -= 360
    # Convert longitudes less than -180 to positive values
    df.loc[df['lon'] < -180, 'lon'] += 360
    x, y = df['lon'], df['lat']
    # split the track into segments that dont cross the dateline
    xdiff = np.diff(x)
    wrapidx = np.where(np.abs(xdiff) > 180)[0] + 1
    segments = np.split(df, wrapidx)
    
    # Get a colormap object
    cmap = plt.get_cmap('hsv')
    
    # Plot each segment separately
    for i, seg in enumerate(segments):
        x, y = seg['lon'], seg['lat']
        IMO = seg['IMO'].iloc[0] # get the value of 'IMO' for this segment
        color_index = hash(str(IMO)) % 256 # generate a unique color index based on IMO value
        color = cmap(color_index / 256) # get the color for this index
        plt.plot(x, y, linewidth=0.1, transform=ccrs.Geodetic(), c="k", zorder=1)
        # plt.scatter(df.lon, df.lat, marker="x", c="k", s=3, linewidth=0.4, zorder=3)


#### Ramer-Douglas-Peuker Algorithm

In [ ]:
%matplotlib inline
rdp_ships_list = []
ship_data_list = []
df_rdp = pd.DataFrame()
ship_data_dict = {}

plt.figure(figsize=(6,6), dpi=400)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor="lightgrey")
gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, 45), ylocs=range(-90, 90, 45))
# # ax.coastlines()
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 8, 'color': 'k'}
gl.ylabel_style = {'size': 8, 'color': 'k'}
gl.xpadding = 15 
gl.ypadding = 10 


for imo, row in df_ships.iterrows():
    # print(imo)
    row_group = row["data"].sort_values(['IMO', "datetime"])
        
    # group same set of betw_port
    row_group['betw_port_group'] = ((row_group['betw_port'] != row_group['betw_port'].shift()) |
                              (row_group['betw_port'] == -1) |
                              (row_group['betw_port'].shift() == -1)).cumsum()
   
    ship_df = pd.DataFrame()

    for i, group in row_group.groupby("betw_port_group"):
        
        # dont apply rdp if ship is in port
        if group["port_id"].notnull().all():
        
            if len(group) > 1:
                # keep only the first and last rows
                group = group.iloc[[0, -1], :]
                ship_df = pd.concat([ship_df, group], ignore_index=True)
                rdp_ships_list.append(group[["lat", "lon"]]) # for clustering

            else:
                ship_df = pd.concat([ship_df, group], ignore_index=True)
                rdp_ships_list.append(group[["lat", "lon"]]) # for clustering


        else: # apply rdp if not in port
            ais_data_coords = group[["lat", "lon"]]
            # rdp_indices = rdp_mod(ais_data_coords.to_numpy(), epsilon=0.08) # run rdp algorithm
            eps = 10
            rdp_indices = rdp_idxs(ais_data_coords.to_numpy(), epsilon=eps) # run rdp algorithm

            rdp_data = group.iloc[rdp_indices]
            ship_df = pd.concat([ship_df, rdp_data], ignore_index=True)
            rdp_ships_list.append(rdp_data[["lat", "lon"]]) # for clustering
            # plot_wrapping2(rdp_data)
            # plt.scatter(rdp_data.lon, rdp_data.lat, s=0.4, linewidth=0.1, c="k", zorder=2)
            # plt.scatter(ais_data_coords.lon, ais_data_coords.lat, s=2, zorder=1)


    ship_data_dict[imo] = ship_df
    
df_rdp = pd.DataFrame.from_dict(ship_data_dict, orient='index')
df_rdp = df_rdp.rename(columns={0: 'data'})
# plt.title(eps)
# plt.show()

In [ ]:
# rdp_all_list = np.concatenate(rdp_ships_list)


# plt.figure()
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.scatter(rdp_all_list[:, 1], rdp_all_list[:, 0], s=0.5, c="k")


#### Clustering RDP Points to form Nodes

In [ ]:
# rdp_all_list = np.concatenate(rdp_ships_list)
# neighbors = NearestNeighbors(n_neighbors=2)
# neighbors_fit = neighbors.fit(rdp_all_list)
# distances, indices = neighbors_fit.kneighbors(rdp_all_list)

# distances = np.sort(distances, axis=0)
# distances = distances[:,1]

# i = np.arange(len(distances))
# knee = KneeLocator(i, distances, S=1, curve='convex', direction='increasing', interp_method='polynomial')

# knee.plot_knee(figsize=(4, 4))
# plt.xlabel("Points")
# plt.ylabel("Distance")

# opt_eps = distances[knee.knee]
# print(opt_eps)

In [ ]:
# # rdp_all_list = np.concatenate(rdp_ships_list)

# # cluster RDP points using DBSCAN
# eps = 0.14
# min_samples = 3

# # dbscan = DBSCAN(eps=eps, min_samples=min_samples)
# # dbscan = DBSCAN(eps=0.18, min_samples=3)

# # dbscan = DBSCAN(eps=opt_eps, min_samples=3)

# eps = 0.18
# min_samples = 20

# # dbscan = DBSCAN(eps=0.2, min_samples=3)
# dbscan = DBSCAN(eps=eps, min_samples=min_samples)


# labels = dbscan.fit_predict(rdp_all_list)

# # ------------------------- Centroid of Cluster -------------------------------

# num_clusters = len(set(labels))
# clusters = pd.DataFrame([rdp_all_list[labels == n] for n in range(num_clusters)], columns = ['value'])
# clusters['Length'] = clusters.value.apply(lambda x: len(x))
# clusters = clusters[clusters.Length > 0]
# clusters = clusters['value']
# centermost_points_1 = clusters.map(get_centermost_point)

# lats, lons = zip(*centermost_points_1)
# intermediary_nodes = pd.DataFrame({'lat':lats, 'lon':lons})
# intermediary_nodes["node_id"] =  [n for n in range(num_clusters-1)] 

#### DBSCAN

In [ ]:
# # rdp_all_list = np.concatenate(rdp_ships_list)
# # # nodes_clustered, node_labels = DBSCAN_km(rdp_all_list, km=10, min_points=10, labels=True)

# # #%% PLOTTING DBSCAN

# plt.figure(figsize=(5,5), dpi=400)
# ax = plt.axes(projection=ccrs.PlateCarree())


# region = "CH"

# if region == "CH":
#     ax.set_extent([80, 160, -30, 50], crs=ccrs.PlateCarree())
# elif region =="EU":
#     ax.set_extent([-20, 50, 20, 75], crs=ccrs.PlateCarree())
# elif region == "US":
#     ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
# elif region == "ME":
#     ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
# elif region == "AF":
#     ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())


# ax.add_feature(cfeature.LAND, facecolor="lightgrey")

# for label in np.unique(labels):
#     mask = (labels == label)
#     x = rdp_all_list[:, 1][mask]
#     y = rdp_all_list[:, 0][mask]

#     if label == -1: # ignore -1 labels (noise)
#         plt.scatter(x, y, s=3, edgecolors="gray", facecolors='none', linewidth=0.2)

#     else:
#         plt.scatter(x, y, s=2)


# ax.scatter(intermediary_nodes.lon, intermediary_nodes.lat, c='k', edgecolor='None', marker = "x", s=20, linewidth=1, zorder=3)
# # ax.scatter(ports_clustered.lon, ports_clustered.lat, c="k", s=20, marker="x", linewidth=0.5)


# plt.title(f"Eps = {eps}, MinPts = {min_samples}")


# # ax.scatter(nodes_clustered.lon, nodes_clustered.lat, c='k', edgecolor='None', marker = "x", alpha=0.7, s=20, zorder=3)
# # ax.scatter(df.lon, df.lat, c='k', edgecolor='None', marker = "x", alpha=0.7, s=0.1, zorder=3)
# # ax.scatter(rdp_all_list[:, 1], rdp_all_list[:, 0], c='r', edgecolor='None', marker = "x", alpha=0.9, s=0.2, zorder=3)
# # ax.scatter(ports_clustered.lon, ports_clustered.lat, c='limegreen', edgecolor='None', marker = "x", alpha=0.7, s=15, zorder=3)




#### HDBSCAN

In [ ]:
rdp_all_list = np.concatenate(rdp_ships_list)

In [ ]:
rdp_all_list = np.concatenate(rdp_ships_list)
import hdbscan

# %matplotlib inline
def HDBSCAN_(coords, min_cluster_size=10, min_points=2, labels=False, eps=False):
    """returns a dataframe of cluster coordinates and cluster ids formed by HDBSCAN"""
    if eps:
        eps = eps/6371
        hdb = hdbscan.HDBSCAN(min_cluster_size, min_points, metric='haversine', cluster_selection_epsilon=eps, cluster_selection_method='eom').fit(np.radians(coords))

    else:
        hdb = hdbscan.HDBSCAN(min_cluster_size, min_points, metric='haversine').fit(np.radians(coords))

    cluster_labels = hdb.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.DataFrame([coords[cluster_labels == n] for n in range(num_clusters)], columns = ['value'])
    clusters['Length'] = clusters.value.apply(lambda x: len(x))
    clusters = clusters[clusters.Length > 0]
    clusters = clusters['value']

    centermost_points = clusters.map(get_centermost_point)
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({'lat':lats, 'lon':lons})
    
    coords_clustered = np.apply_along_axis(lambda row: coords[(coords[:,0]==row[0]) & (coords[:,1]==row[1])][0], axis=1, arr=rep_points)
    coords_clustered = pd.DataFrame(coords_clustered, columns=['lat', 'lon'])
    coords_clustered = coords_clustered.reset_index(drop=True) 
    coords_clustered.index += 1 # start index from 1
    coords_clustered['port_id'] = coords_clustered.index

    cmap = plt.cm.get_cmap('turbo', num_clusters)

    if labels:
        return coords_clustered, cluster_labels

    else:
        return coords_clustered


min_points = 30
min_cluster_size = 30
eps=25



nodes_clustered, node_labels = HDBSCAN_(rdp_all_list, min_cluster_size=min_cluster_size, min_points=min_points, labels=True, eps=eps)
nodes_clustered = nodes_clustered.rename(columns={'port_id': 'node_id'})






In [ ]:
%matplotlib inline

plt.figure(figsize=(8,8), dpi=400)
plt.figure(figsize=(5,5), dpi=400)
ax = plt.axes(projection=ccrs.PlateCarree())

region = "EU"

if region == "CH":
    ax.set_extent([80, 160, -30, 50], crs=ccrs.PlateCarree())
elif region =="EU":
    ax.set_extent([-12, 37, 30, 65], crs=ccrs.PlateCarree())
elif region == "US":
    ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
elif region == "ME":
    ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
elif region == "AF":
    ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())



# ax.coastlines()
ax.add_feature(cfeature.LAND, facecolor="lightgrey")
gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, 10), ylocs=range(-90, 90, 10))
# # ax.coastlines()
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 8, 'color': 'k'}
gl.ylabel_style = {'size': 8, 'color': 'k'}
gl.xpadding = 15 
gl.ypadding = 10 
for label in np.unique(node_labels):
    mask = (node_labels == label)
    x = rdp_all_list[:, 1][mask]
    y = rdp_all_list[:, 0][mask]

    if label == -1: # ignore -1 labels (noise)
        plt.scatter(x, y, s=1, edgecolors="gray", facecolors='none', linewidth=0.2)

    else:
        plt.scatter(x, y, s=1)
# plt.legend()
ax.scatter(nodes_clustered.lon, nodes_clustered.lat, c='k', edgecolor='None', marker = "x", s=10, linewidth=1, zorder=3)
# ax.scatter(ports_clustered.lon, ports_clustered.lat, c="k", s=20, marker="x", linewidth=0.5)


plt.title(f"min_cluster_size = {min_cluster_size}, minPts = {min_points},  eps = {eps}")



#### Database of Nodes (including ports)

In [ ]:
#%%
max_port_id = ports_clustered.port_id.max()


# ------------------- DATABASE OF ALL NODES (INCLUDING PORTS) -----------------
def create_node_df():
    add_port = lambda x: 'P' + str(x) # add a letter in front of each value
    all_ports = ports_clustered[["lat", "lon"]]
    all_ports['id'] = ports_clustered['port_id'].apply(add_port)

    add_node = lambda x: 'N' + str(x + max_port_id) # add a letter in front of each value

    all_int_nodes = nodes_clustered[["lat", "lon"]]
    all_int_nodes['id'] = nodes_clustered['node_id'].apply(add_node)

    all_nodes = pd.concat([all_ports, all_int_nodes])
    all_nodes = all_nodes.reset_index(drop=True)

    return all_nodes


# IF VALUE of port is within 5km of value of node, delete node.

def filter_nodes_within_distance(df, distance):
    # split the dataframe into two dataframes, based on id prefix
    df_n = df[df['id'].str.startswith('N')]
    df_p = df[df['id'].str.startswith('P')]
    
    # distances between each node with 'N' and port with 'P'
    distances = haversine_np(df_n['lon'].values[:, np.newaxis], df_n['lat'].values[:, np.newaxis],
                             df_p['lon'].values[np.newaxis, :], df_p['lat'].values[np.newaxis, :])
    
    # minimum distance for each node N
    min_distances = np.min(distances, axis=1)
    
    # remove nodes N within specified distance a port 
    filtered_nodes = df_n[min_distances > distance]
    
    # get the deleted nodes N and their closest ports
    deleted_nodes = df_n[min_distances <= distance]
    
    df_p_reset = df_p.reset_index()
    closest_ports = df_p_reset.loc[np.argmin(distances, axis=1), :][min_distances <= distance]
    deleted_nodes['closest_port'] = closest_ports['id'].values    
    result = pd.concat([filtered_nodes, df_p])
    
    return result, deleted_nodes

all_nodes = create_node_df()
all_nodes_f, deleted_nodes = filter_nodes_within_distance(all_nodes, 5)


#### Nodes Visited for each record

In [ ]:
#%% Nodes visited

df_rdp = pd.DataFrame.from_dict(ship_data_dict, orient='index')
df_rdp = df_rdp.rename(columns={0: 'data'})  


loop_val = 0
temp_arr = []


def remove_decimal(df, column_name):
    """
    removes decimal point and following characters in a string
    eg:
    P12.0 becomes P12
    """

    df[column_name] = np.where(~df[column_name].isna(),
                               df[column_name].astype(str).str.replace(r'\.\d+', ''),
                               df[column_name])
    
    return df


def create_betw_node(df, node_column_name):
    """
    This function calculates the 'from_node' and 'to_node' columns of a DataFrame based on a given 'node' column.
    """

    df["node_value"] = df[node_column_name].str[1:].astype(float) # extract node value
    df["node_fwd"] = df["node_value"].ffill()
    df["node_bwd"] = df["node_value"].bfill()
    df["subtract1"] = df["node_bwd"] - df["node_value"]
    df["subtract1"] = df["subtract1"].replace(np.NaN, 1)
    df["subtract1"] = df["subtract1"].replace(0, np.NaN)
    df["from_node"] = df["node_fwd"] * df["subtract1"]
    df["to_node"] = df["node_bwd"] * df["subtract1"]
    
    mask = df["from_node"].notna()
    df.loc[mask, "from_node"] = ["N"+str(node) if node > (max_port_id) else "P"+str(node) for node in df.loc[mask, "from_node"]]
    df.loc[mask, "to_node"] = ["N"+str(node) if node > (max_port_id) else "P"+str(node) for node in df.loc[mask, "to_node"]]
    
    remove_decimal(df, 'from_node')
    remove_decimal(df, 'to_node')
    
    df["betw_node"] = list(zip(df["from_node"], df["to_node"]))
    df.drop(columns=["subtract1", "node_fwd", "node_bwd", "from_node", "to_node"], inplace=True)

    return df


for imo, row in df_rdp.iterrows():

    #---------------------- SET COLUMN WITH NODE VALUE ------------------------
    node_values = node_labels[loop_val:(loop_val + len(row.data))]
    node_values = np.where(node_values == -1, np.nan, node_values + 1 + max_port_id)    
    node_values = ['N' + str(val) if not np.isnan(val) else np.nan for val in node_values] # add N in front of node number
    node_values = [val.split('.')[0] if not pd.isna(val) else val for val in node_values]
    
    row_data = row.data.assign(node=node_values) # add node values as column "node"
    loop_val += len(row.data)
    
    #--------------- IN PORT, SET PORT AS NODE IN NODE COLUMN ----------------
    
    # add port_id with "P" prefix to node column for non-nan port_id values
    row_data["node"] = np.where(row_data["port_id"].isna(), row_data["node"], "P" + row_data["port_id"].astype(str))
        
    # remove decimal points from node column
    row_data = remove_decimal(row_data, 'node')
      
    #------------------------ BETWEEN NODES COLUMN ----------------------------
    row_data = create_betw_node(row_data, "node")   

    #--------------------- REPLACE (nan, nan) with (node, node)----------------  

    # replace nan values for node with tuple of node id it is at
    row_data['betw_node'] = np.where(row_data['betw_node'].apply(lambda x: pd.isna(x[0]) and pd.isna(x[1])),  # condition
                                    row_data['node'].apply(lambda x: (x, x)),  # new value
                                    row_data['betw_node'])  # existing value
    
    #------------------CLOSEST POINT TO CENTROID WITHIN CLUSTER ---------------
    new_data = pd.DataFrame()
    row_data['betw_node_group'] = (row_data['betw_node'] != row_data['betw_node'].shift()).cumsum() # group same set of betw_port

    for i, group in row_data.groupby("betw_node_group"):
        node_value = group.iloc[0]["node"]

        if not pd.isna(node_value):
        
            if node_value.startswith("N"): # closest point to centroid of node
                # print(node_value)            
                node_lat, node_lon = all_nodes.loc[all_nodes["id"] == node_value, ["lat", "lon"]].values[0]
                tree = BallTree(group[['lat', 'lon']].values, metric='haversine')
                    
                # find the index of the nearest neighbor for each point in the group
                _, indices = tree.query([[node_lat, node_lon]], k=1)
                
                nearest_neighbor = group.iloc[indices[0]]                       
                nearest_neighbor['at_node'] = nearest_neighbor['node'].copy() 
                group['at_node'] = nearest_neighbor['at_node']
                group['at_node'] = group['at_node'].fillna(value=np.nan)
                group.drop(columns=["betw_node", "betw_node_group"], inplace=True)
                
            else: # if node is port node
                group['at_node'] = group['node'].copy()
                        
        new_data = pd.concat([new_data, group], ignore_index=True)  
        
    new_data = create_betw_node(new_data, "at_node")  
    
    # replace nan values for port with tuple of port id it is in
    new_data['betw_node'] = np.where(new_data['betw_node'].apply(lambda x: pd.isna(x[0]) and pd.isna(x[1])),  # condition
                                    new_data['at_node'].apply(lambda x: (x, x)),  # new value
                                    new_data['betw_node'])  # existing value
    
    # print(new_data)

    result = new_data['betw_node'].apply(lambda x: x if (not str(x[0])[0].isalpha()) or (not str(x[1])[0].isalpha()) else None).dropna()
    if not result.empty:
        print(imo)
        print(result)
        print(new_data)

    new_data['betw_node_group'] = (new_data['betw_node'] != new_data['betw_node'].shift()).cumsum() # group same set of betw_port 

    #------------------- OUTER COLUMN WITH ALL NODES VISITED ------------------
    
    nodes_visited_col = new_data['at_node'].tolist()
    nodes_visited_nonan = [x for x in nodes_visited_col if not pd.isna(x)] # remove nan values first
    nodes_visited_unique = [x[0] for x in groupby(nodes_visited_nonan)] # remove consecutive values that are the same
    
    #recalcualte distances and time differences after applying rdp
    new_data = calc_distances(new_data)


    record = {'IMO': imo, 'data': new_data, "nodes_visited": nodes_visited_unique}
    temp_arr.append(record)     


df_rdp_nodes = pd.DataFrame.from_records(temp_arr, index='IMO')    

In [ ]:
def create_ship_network(df):
    result_dict = {}
    for imo, row in df.iterrows():
        display(row.data[['IMO', 'lat', 'lon', 'sog', 'datetime', 'dist', 'time_diff', 'port_id', 'betw_port', 'node', 'betw_node']])
        break

create_ship_network(df_rdp_nodes)


In [ ]:
df_rdp_nodes = pd.read_pickle("df_rdp_nodes.pkl")
def create_ship_network(df):
    result_dict = {}
    for imo, row in df.iterrows():
        data_frames = []
        prev_node_comb = 0
        for i, port_group in row.data.groupby("betw_port_group"):

            port_comb = port_group.iloc[0]["betw_port"] 
            print("=============================================================================")
            print("---------------------NEW PORT COMB-", port_comb, "--------------------")
            print("port_group", port_group)
            print("-----------------------------------------------------------------------------")

            data_df = pd.DataFrame()
            data_df["betw_port"] = [port_comb]
            node_data_df = pd.DataFrame(columns=["betw_node", "time_diff", "dist", "start_month"]) 

            first_iter = True
            for j, node_group in port_group.groupby("betw_node_group"):
                node_comb = node_group.iloc[0]["betw_node"] 
                # print(node_comb)
                at_node_value = node_group.iloc[0]["at_node"] 

                next_group = port_group[port_group["betw_node_group"] == j+1]
                prev_group = port_group[port_group["betw_node_group"] == j-1]

                # print("Previous = ", prev_group)
                start_month = node_group.iloc[0]["datetime"].month
                
                if node_comb[0] == node_comb[1] and node_comb[1].startswith("P"):
                    total_time = port_group.iloc[-1]["datetime"] - port_group.iloc[0]["datetime"]
                    # total_dist = port_group["dist"].sum()
                    total_dist = 0 #(in port)
                    
                    node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                    node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                    # print("Added -a): ", node_comb)
                    last_added = node_comb
                    break

                else: 
                    if prev_group.empty and next_group.empty: # eg ('P29', 'P101') when at P101 (going only between two ports)
                        #add port

                        total_time = row.data[row.data["betw_port_group"] == i + 1].iloc[0]["datetime"] - row.data[row.data["betw_port_group"] == i - 1].iloc[0]["datetime"]
                        total_dist = node_group["dist"].sum() + row.data[row.data["betw_port_group"] == i + 1].iloc[0]["dist"]
                        node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                        node_data_df = node_data_df.append(node_data_row, ignore_index=True)
                        # print("Added a): ", node_comb)
                        last_added = node_comb
                    
                    elif prev_group.empty and not next_group.empty:
                        

                        if pd.isna(at_node_value):
                            total_time = node_group.iloc[0]["time_diff"] 
                            total_dist = node_group.iloc[0]["dist"] 
                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added b)", node_comb) # between node value
                            last_added = node_comb

                        else:
                            
                            new_node_comb = ("P" + str(int(port_comb[0])), at_node_value)
                            total_time = node_group.iloc[0]["time_diff"] 
                            total_dist = node_group.iloc[0]["dist"] 
                            node_data_row = {'betw_node': new_node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added c)" , new_node_comb)
                            # if it is a node=node, time spent at node is zero, distance is zero as well
                            total_time = datetime.timedelta()  # last 
                            total_dist = 0 # last
                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)
                            # print("Added c)" , node_comb)
                            
                            last_added = node_comb
                    
                    
                    elif not prev_group.empty and not next_group.empty:
                        if last_added[1] == at_node_value: 

                            total_time = datetime.timedelta()
                            total_dist = 0
                            
                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added d)" , node_comb)
                            last_added = node_comb

                        elif not pd.isna(at_node_value):
                            new_node_comb = (last_added[1], at_node_value)
                            total_time = node_group.iloc[0]["time_diff"] 
                            total_dist = node_group.iloc[0]["dist"] 
                            node_data_row = {'betw_node': new_node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added e)", new_node_comb)
                            
                            total_time = datetime.timedelta()
                            total_dist = 0

                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)
                            # print("Added e)", node_comb)
                            last_added = node_comb

                        else: # nan value 
                            total_time = next_group.iloc[0]["datetime"] - prev_group.iloc[-1]["datetime"]
                            total_dist = node_group.dist.sum() + next_group.iloc[0]["dist"]  

                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)
                        
                            # print("Added f) ", node_comb)
                            last_added = node_comb

                    elif not prev_group.empty and next_group.empty:
                        if last_added[1] == at_node_value: 

                            total_time = datetime.timedelta()
                            total_dist = 0

                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added g)" , node_comb)

                            if node_comb[1].startswith("N"):
                                new_node_comb = (at_node_value, "P" + str(int(port_comb[1])))
                                total_time = row.data[row.data["betw_port_group"] == i + 1].iloc[0]["time_diff"] # NEXT GROUP VALUE
                                total_dist = row.data[row.data["betw_port_group"] == i + 1].iloc[0]["dist"] 
                                node_data_row = {'betw_node': new_node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                                node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                                # print("Added g)" , new_node_comb)
                                last_added = node_comb

                            last_added = node_comb

                        elif not pd.isna(at_node_value):
                            new_node_comb = (last_added[1], at_node_value)
                            total_time = node_group.iloc[0]["time_diff"] 
                            total_dist = node_group.iloc[0]["dist"] 
                            node_data_row = {'betw_node': new_node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added h)", new_node_comb)

                            total_time = datetime.timedelta()
                            total_dist = 0
                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added h)" , node_comb)

                            if node_comb[1].startswith("N"):
                                new_node_comb = (at_node_value, "P" + str(int(port_comb[1])))
                                total_time = row.data[row.data["betw_port_group"] == i + 1].iloc[0]["time_diff"] # NEXT GROUP VALUE
                                total_dist = row.data[row.data["betw_port_group"] == i + 1].iloc[0]["dist"] 
                                node_data_row = {'betw_node': new_node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                                node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                                # print("Added h)" , new_node_comb)
                            last_added = node_comb

                        else: # N601 -> NaN -> P4 (last value is nan)
                            total_time = row.data[row.data["betw_port_group"] == i + 1].iloc[0]["datetime"] - prev_group.iloc[-1]["datetime"] 
                            total_dist = port_group["dist"].sum() + row.data[row.data["betw_port_group"] == i + 1].iloc[0]["dist"]
                            node_data_row = {'betw_node': node_comb, "time_diff": total_time, "dist": total_dist, "start_month": start_month} # Define a dictionary with the values for each column
                            node_data_df = node_data_df.append(node_data_row, ignore_index=True)

                            # print("Added i) ", node_comb)
                            last_added = node_comb
                
                # else: 
                #     print("!!!!!!!!!MISSING!!!!!!!!!!", node_comb)

            data_df["node_data"] = [node_data_df]
            data_frames.append(data_df)
        result_dict[imo] = pd.concat(data_frames, ignore_index=True)
    return pd.DataFrame([result_dict])


df_ship_network_test = create_ship_network(df_rdp_nodes)
df_ship_network_T = df_ship_network_test.T

#### Number of Ports Visited per Ship

In [ ]:

for imo, row in df_ship_network_T.iterrows():
    betw_port_level = row.T[0]
    ports_only = betw_port_level[betw_port_level["betw_port"].apply(lambda x: x[0] == x[1])]
    # display(ports_only)
    total = len(ports_only)
    # display(ports_only["betw_port"].unique())
    num_unique_betw_ports = ports_only["betw_port"].nunique()
    # print("Ship:", imo, "unique ports:", num_unique_betw_ports, "total:  ", total)
    df_ship_network_T.at[imo, "total_ports"] = total
    df_ship_network_T.at[imo, "unique_ports"] = num_unique_betw_ports
    df_ship_network_T.at[imo, "repeatability"] = total - num_unique_betw_ports


In [ ]:
percentage_nonzero = (len(df_ship_network_T[df_ship_network_T['repeatability'] == 0]) / len(df_ship_network_T)) * 100
print(percentage_nonzero)

## SHIP DETAILS

In [ ]:
ship_details = pd.read_csv(folder_loc + r"\bulk_carrier_data\ship_details.csv")
ship_details_all = pd.read_csv(folder_loc + r"\bulk_carrier_data\ship_details_all.csv")

ship_details["IMO Number"] = ship_details["IMO Number"].astype(str)
ship_details_all["IMO Number"] = ship_details_all["IMO Number"].astype(str)


In [ ]:
df_ship_network_T['IMO'] = df_ship_network_T.index
df_ship_network_T['IMO'] = df_ship_network_T['IMO'].astype(str)

# extract the IMO number from the "IMO" column
df_ship_network_T["IMO_number"] = df_ship_network_T["IMO"].str.split("_").str[0]

# create a dictionary mapping IMO numbers to capacities from the other dataframe
imo_capacity_dict = dict(zip(ship_details_all["IMO Number"].str.split("_").str[0], ship_details_all["Deadweight"]))

# map the IMO number to the corresponding capacity value
df_ship_network_T["Deadweight"] = df_ship_network_T["IMO_number"].map(imo_capacity_dict)



In [ ]:
# ships that arent in amys data
a1 = df_ship_network_T["IMO_number"]
a2 = ship_details["IMO Number"]
df_ship_network_T["IMO_number"] = df_ship_network_T["IMO"].str.split("_").str[0]
not_in_ship_details = ~df_ship_network_T["IMO_number"].isin(ship_details["IMO Number"])
df_ship_network_T_not_in_ship_details = df_ship_network_T[not_in_ship_details]


In [ ]:
df_ship_network_T_not_in_ship_details = df_ship_network_T_not_in_ship_details.sort_values(by="Deadweight")

imo_number = df_ship_network_T_not_in_ship_details["IMO_number"]
total_ports = df_ship_network_T_not_in_ship_details["total_ports"]
unique_ports = df_ship_network_T_not_in_ship_details["unique_ports"]
dwt = df_ship_network_T_not_in_ship_details["Deadweight"]

fig, ax = plt.subplots(figsize=(6,3), dpi=200)
ax.bar(imo_number, total_ports, label="Total Destinations", color="k")
ax.bar(imo_number, unique_ports, label="Unique Destinations", color="steelblue")
ax.set_xlabel("Ship (increasing deadweight)")
ax.set_ylabel("Number of Destinations")
ax.legend()
ax.set_xticklabels("")
ax.set_xticks("")
plt.show()

### Reverse Geocoding (Mapbox)

In [ ]:
import requests
from geopy.distance import distance
from sklearn.neighbors import BallTree
import pycountry

country_code_dict = {}
for country in pycountry.countries:
    country_code_dict[country.alpha_2] = country.name

wpi_ports = gpd.read_file(r'C:\Users\Nefelie\OneDrive - University of Southampton\Uni\3rd Year\Individual Project\other\World_Port_Index\World_Port_Index.shp')
tree = BallTree(np.radians(wpi_ports[['LATITUDE', 'LONGITUDE']].values), leaf_size=15, metric='haversine')

access_token = 'INSERT OWN API ACCESS TOKEN'
geocoding_endpoint = 'mapbox.places'

def geopy_distance(coords1, coords2):
    return distance(coords2, coords1)

for index, row in all_nodes.iterrows():
    lat, lon = row['lat'], row['lon']
    url = f"https://api.mapbox.com/geocoding/v5/{geocoding_endpoint}/{lon},{lat}.json?access_token={access_token}&types=country"
    response = requests.get(url)
    data = response.json()
    features = data.get('features', [])
    if len(features) > 0:
        country = features[0]['text']
        if country == 'Russian Federation':
            country = 'Russia'

    else:
        # country = None

        dist, ind = tree.query([np.radians([lat, lon])], k=1)
        closest_node = wpi_ports.iloc[ind[0][0]]
        closest_distance = distance((lat, lon), (closest_node['LATITUDE'], closest_node['LONGITUDE'])).km
        # print(closest_distance)
        if closest_distance <= 2000:
            country = closest_node['COUNTRY']
            # print(country)
            country = pycountry.countries.get(alpha_2=country).name
            # print(country)
            if country == 'Russian Federation':
                country = 'Russia'


        else:
            print(row["id"], closest_distance)
            country = None

    all_nodes.at[index, 'country'] = country

p_rows = all_nodes[all_nodes['id'].str.startswith('P')]  # Filter rows where ID starts with 'P'
p_nan_count = p_rows['country'].isna().sum()  # Count the number of NaN values in the column
p_percent_nan = p_nan_count / len(p_rows) * 100  # Calculate the percentage of NaN values
print(p_percent_nan)

In [ ]:
p_nodes = all_nodes[all_nodes['id'].str.startswith('P')]

num_countries = len(p_nodes['country'].unique())

print(f"Number of countries in p_nodes starting with 'P': {num_countries}")
top_port_countries = pd.DataFrame({'country': p_nodes['country'].value_counts().head(10).index, 'no_ports': p_nodes['country'].value_counts().head(10).values})
top_port_countries = top_port_countries.reset_index(drop=True)
top_port_countries.index = top_port_countries.index + 1
display(top_port_countries)


In [ ]:
print(p_nan_count)

In [ ]:
def crs_plot(world_bw=False, world_color=False, t=True, b=True, l=True, r=True, x_spac=45, y_spac=45):
    """Generate a plot in the PlateCarree projection"""
    plt.figure(dpi=400)
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
    gl = ax.gridlines(draw_labels=True, alpha=0.2, color="k", linestyle='--', linewidth=0.3, xlocs=range(-180, 180, x_spac), ylocs=range(-90, 90, y_spac))
    gl.top_labels = t
    gl.right_labels = r
    gl.bottom_labels = b
    gl.left_labels = l
    gl.xlabel_style = {'size': 8, 'color': 'k'}
    gl.ylabel_style = {'size': 8, 'color': 'k'}
    gl.xpadding = 15 
    gl.ypadding = 10 

    if world_color:
        ax.add_feature(cfeature.LAND)
        ax.add_feature(cfeature.OCEAN)
        ax.add_feature(cfeature.BORDERS, linestyle='-', alpha=.5, linewidth=0.1)
    
    elif world_bw:
        ax.add_feature(cfeature.LAND, color="lightgrey")

    return plt, ax


## Ships MultiGraph

In [ ]:
G_ships_multi = nx.MultiGraph()

for imo, row in df_ship_network_T.iterrows():
    row_data = row.T.iloc[0]
    for _, sub_row in row_data.iterrows():
        port_comb = sub_row[0]
        for _, node_group in sub_row[-1].iterrows():
            node_comb = node_group['betw_node']
            G_ships_multi.add_edge(node_comb[0], node_comb[1],
                                   time=node_group.loc['time_diff'], 
                                   dist=node_group.loc['dist'], 
                                   start_month=node_group.loc['start_month'],
                                   ship_imo=imo, 
                                   port_comb=port_comb)

for u, v, edge_attrs in G_ships_multi.edges(data=True):
    if u == v:  # Check if self-loop edge
        if u.startswith("P"):
            node_type="P"
        else:
            node_type="N"
        lat = all_nodes.loc[all_nodes['id'] == u, 'lat'].iloc[0]
        lon = all_nodes.loc[all_nodes['id'] == u, 'lon'].iloc[0]
        country = all_nodes.loc[all_nodes['id'] == u, 'country'].iloc[0]
        G_ships_multi.add_node(u, lat=lat, lon=lon, country=country, start_month=edge_attrs["start_month"], ship_imo=edge_attrs["ship_imo"], time=edge_attrs["time"], node_type=node_type)

# for i, row in all_nodes.iterrows():
#     if row.id.startswith("N"):
#         node_type = "N"
#     elif row.id.startswith("P"):
#         node_type = "P"
#     G_ships_multi.add_node(row.id, lat=row['lat'], lon=row['lon'], node_type=node_type)

In [ ]:
df_G_ships_multi = nx.to_pandas_edgelist(G_ships_multi)
display(df_G_ships_multi[105:115])

In [ ]:
G_ships_multi.remove_edges_from(nx.selfloop_edges(G_ships_multi))
pos = {node: (G_ships_multi.nodes[node]['lon'], G_ships_multi.nodes[node]['lat']) for node in G_ships_multi.nodes}
plt, ax = crs_plot(world_bw=True, world_color=True)
plt.scatter(df.lon, df.lat, s=0.05, linewidth=0.01, alpha=0.5, c="lavender", zorder=1)
nx.draw_networkx_nodes(G_ships_multi, pos, node_size=0.1, node_color='k', ax=ax)
nx.draw_networkx_edges(G_ships_multi, pos, edge_color='k', alpha=0.5, width=0.05, ax=ax)

plt.show()

#### Example of Anchored Ship

In [ ]:
print(df_ships.loc["9498896"]["data"]["lon"])
plt.scatter(df_ships.loc["9498896"]["data"]["lon"], df_ships.loc["9498896"]["data"]["lat"], s=0.01)

In [ ]:
imo_test = "9498896"

ship_edges = [(u, v, data) for u, v, data in G_ships_multi.edges(data=True) if data['ship_imo']==imo_test]
print(ship_edges)

pos_ships = {node:(data['lon'], data['lat']) for node, data in G_ships_multi.nodes(data=True)}

plt, ax = crs_plot(world_bw=True, world_color=False, x_spac=1, y_spac=1)

plt.scatter(df_ships.loc["9498896"]["data"]["lon"], df_ships.loc["9498896"]["data"]["lat"], s=0.01, label="ship track")
nx.draw_networkx_edges(G_ships_multi, pos_ships, edgelist=ship_edges, edge_color='r', width=1.0, alpha=0.7, label="edges")

ax.set_extent([-16, -14, 10, 12], crs=ccrs.PlateCarree())
plt.legend(markerscale=10)
plt.title(f"IMO {imo_test}")
plt.show()

### Ports graph

#### MultiGraph

In [ ]:
G_ports_multi = nx.MultiGraph()

for imo, row in df_ship_network_T.iterrows():
    row_data = row.T.iloc[0]
    port_comb_dict = {}
    for _, sub_row in row_data.iterrows():
        port_comb = sub_row[0]
        port_comb = tuple(["P" + str(int(val)) for val in port_comb])
        # If this port_combination hasn't been seen before, add it to the dictionary
        if port_comb not in port_comb_dict:
            port_comb_dict[port_comb] = {'time': datetime.timedelta(0), 'dist': 0, 'imo': set()}

        for _, node_group in sub_row[-1].iterrows():
            node_comb = node_group['betw_node']
            port_comb_dict[port_comb]['time'] += pd.Timedelta(node_group['time_diff'])
            port_comb_dict[port_comb]['dist'] += pd.Series(node_group['dist']).sum()
            port_comb_dict[port_comb]['imo'].update({imo})

    for port_comb, edge_attrs in port_comb_dict.items():
        G_ports_multi.add_edge(port_comb[0], port_comb[1], time=edge_attrs['time'], dist=edge_attrs['dist'], imo=list(edge_attrs['imo']))


# Add self-loop edges as nodes to the graph with the same attributes as the edges
for u, v, edge_attrs in G_ports_multi.edges(data=True):
    if u == v and u.startswith("P"):  # Check if it's a self-loop edge
        lat = all_nodes.loc[all_nodes['id'] == u, 'lat'].iloc[0]
        lon = all_nodes.loc[all_nodes['id'] == u, 'lon'].iloc[0]
        country = all_nodes.loc[all_nodes['id'] == u, 'country'].iloc[0]
        G_ports_multi.add_node(u, lat=lat, lon=lon, country=country)


In [ ]:
# G_ports_multi.remove_edges_from(nx.selfloop_edges(G_ports_multi))
pos = {node: (G_ports_multi.nodes[node]['lon'], G_ports_multi.nodes[node]['lat']) for node in G_ports_multi.nodes}
plt, ax = crs_plot(world_bw=True, world_color=False)
nx.draw_networkx_nodes(G_ports_multi, pos, node_size=1, node_color='k', ax=ax)
nx.draw_networkx_edges(G_ports_multi, pos, edge_color='k', alpha=0.1, width=0.5, ax=ax)
plt.show()

#### DiGraph

In [ ]:
G_ships_ports = nx.Graph() #NOTE SHOULD BE DIRECTED GRAPH
for u, v in G_ports_multi.edges():
    edge_dict = G_ports_multi[u][v]
    no_trips = len(edge_dict)
    dists = [edge_dict[i]['dist'] for i in range(no_trips)]
    time_diffs = [edge_dict[i]['time'].total_seconds() for i in range(no_trips)]

    if len(dists) > 0:
        mean_dist, std_dist = np.mean(dists), np.std(dists)
    else:
        mean_dist, std_dist = np.nan, np.nan

    if len(time_diffs) > 0:
        mean_time, std_time = datetime.timedelta(seconds=np.mean(time_diffs)), datetime.timedelta(seconds=np.std(time_diffs))
    else:
        mean_time, std_time = np.nan, np.nan

    if not G_ships_ports.has_edge(u, v):
        G_ships_ports.add_edge(u, v, no_trips=no_trips, mean_dist=mean_dist, std_dist=std_dist, mean_time=mean_time, std_time=std_time)
    else:
        G_ships_ports[u][v]['no_trips'] = no_trips
        G_ships_ports[u][v]['mean_dist'] = mean_dist
        G_ships_ports[u][v]['std_dist'] = std_dist
        G_ships_ports[u][v]['mean_time'] = mean_time
        G_ships_ports[u][v]['std_time'] = std_time

# for i, row in all_nodes.iterrows():
#     if row.id.startswith("N"):
#         node_type = "N"
#     elif row.id.startswith("P"):
#         node_type = "P"
#     G_ships_ports.add_node(row.id, lat=row['lat'], lon=row['lon'], node_type=node_type)

# Add self-loop edges as nodes to the graph with the same attributes as the edges
for u, v, edge_attrs in G_ships_ports.edges(data=True):
    if u == v and u.startswith("P"):  # Check if it's a self-loop edge
        lat = all_nodes.loc[all_nodes['id'] == u, 'lat'].iloc[0]
        lon = all_nodes.loc[all_nodes['id'] == u, 'lon'].iloc[0]
        country = all_nodes.loc[all_nodes['id'] == u, 'country'].iloc[0]
        mean_time = G_ships_ports[u][v]['mean_time']
        std_time = G_ships_ports[u][v]['std_time']
        G_ships_ports.add_node(u, lat=lat, lon=lon, country=country, mean_time=mean_time, std_time=std_time)


G_ships_ports.remove_edges_from(nx.selfloop_edges(G_ships_ports))
df_G_ships_ports = nx.to_pandas_edgelist(G_ships_ports)

#### Plot

In [ ]:
pos = {node_id: (lon, lat) for node_id, lat, lon in zip(all_nodes['id'], all_nodes['lat'], all_nodes['lon'])}

plt, ax = crs_plot(world_bw=True)
nx.draw_networkx_edges(G_ships_ports, pos, width=0.1)
nx.draw_networkx_nodes(G_ships_ports, pos, node_size=1, node_color="r")
plt.show()

#### Mean time spent in port

In [ ]:
mean_times = {u: G_ships_ports.nodes[u]['mean_time'] for u in G_ships_ports.nodes() if u.startswith("P")}
mean_times_float = np.array([mean_time.total_seconds() for node, mean_time in mean_times.items()])
sizes = 200 * (mean_times_float - mean_times_float.min()) / (mean_times_float.max() - mean_times_float.min())

plt, ax = crs_plot(world_bw=True)

pos = {node: (G_ships_ports.nodes[node]['lon'], G_ships_ports.nodes[node]['lat']) for node in G_ships_ports.nodes()}
nx.draw_networkx_nodes(G_ships_ports, pos, nodelist=[node for node in G_ships_ports.nodes() if node.startswith("P")],
                       node_size=sizes, alpha=0.5, node_color='blue')
nx.draw_networkx_edges(G_ships_ports, pos, alpha=0.1)


In [ ]:
%matplotlib inline

In [ ]:
import cmasher as cmr
import matplotlib.pyplot as plt
import networkx as nx

mean_times = nx.get_node_attributes(G_ships_ports, 'mean_time')
std_times = nx.get_node_attributes(G_ships_ports, 'std_time')

std_times = [std_times[node].total_seconds() / (24*60*60) for node in G_ships_ports.nodes() if node.startswith("P")]  # convert to days
mean_times = [mean_times[node].total_seconds() / (24*60*60) for node in G_ships_ports.nodes() if node.startswith("P")]  # convert to days
sorted_nodes = sorted([(node, std_times[i]) for i, node in enumerate(G_ships_ports.nodes()) if node.startswith("P")], key=lambda x: x[1])
sorted_nodes = [node for node, _ in sorted_nodes]

pos = {node: (G_ships_ports.nodes[node]['lon'], G_ships_ports.nodes[node]['lat']) for node in G_ships_ports.nodes()}

sizes = [std_times[i]*0.5 for i in range(len(G_ships_ports.nodes())) if list(G_ships_ports.nodes())[i].startswith("P")]
node_colors = [mean_times[i] for i in range(len(G_ships_ports.nodes())) if list(G_ships_ports.nodes())[i].startswith("P")]
vmax = 30


cmap = cmr.get_sub_cmap('gnuplot2', 0, 0.95)
vmin = min(node_colors)
cmap_values = plt.Normalize(vmin=vmin, vmax=vmax)(node_colors)
cmap_colors = cmap(cmap_values)
white_nodes = ['white' if node_colors[i]>vmax else cmap_colors[i] for i in range(len(node_colors))]

plt, ax = crs_plot(world_bw=True, world_color=True)

nx.draw_networkx_nodes(G_ships_ports, pos, nodelist=sorted_nodes,
                       node_size=sizes, alpha=1, node_color=white_nodes)

# nx.draw_networkx_edges(G_ships_ports, pos, alpha=0.1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
cbar = plt.colorbar(sm, orientation='horizontal', pad=0.1, aspect=50)
cbar.ax.tick_params(labelsize=8)
cbar.minorticks_on()
cbar.set_label('Mean Ship Waiting and Berthing Time (days)')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

mean_times = nx.get_node_attributes(G_ships_ports, 'mean_time')
mean_times = [mean_times[node].total_seconds() / (24 * 60 * 60) for node in G_ships_ports.nodes() if node.startswith("P")]
betweenness = nx.degree_centrality(G_ships_ports)


x = [betweenness[node] for node in G_ships_ports.nodes() if node.startswith("P")]
y = mean_times
plt.figure(dpi=200, figsize=(5,3))
plt.scatter(x, y, s=2, c="k")
plt.xlabel('Betweenness Centrality')
plt.ylabel('Mean Time in Port (days)')
plt.show()



#### Country Graph

In [ ]:
from collections import defaultdict

G_country = nx.Graph()

# group the ports by country
ports_by_country = defaultdict(list)
for node_id, node_data in G_ships_ports.nodes(data=True):
    country = node_data["country"]
    ports_by_country[country].append(node_id)

# create a node for each country
for country, ports in ports_by_country.items():
    G_country.add_node(country, no_trips=0)

# create edges between countries based on the connections between their ports
for u, v, edge_data in G_ships_ports.edges(data=True):
    u_data = G_ships_ports.nodes[u]
    v_data = G_ships_ports.nodes[v]
    u_country = u_data["country"]
    v_country = v_data["country"]
    if u_country != v_country:
        if G_country.has_edge(u_country, v_country):
            G_country[u_country][v_country]["no_trips"] += edge_data["no_trips"]
            G_country[u_country][v_country]["mean_dist"] += edge_data["mean_dist"]
            G_country[u_country][v_country]["std_dist"] += edge_data["std_dist"]
            G_country[u_country][v_country]["mean_time"] += edge_data["mean_time"]
            G_country[u_country][v_country]["std_time"] += edge_data["std_time"]
        else:
            G_country.add_edge(u_country, v_country, **edge_data)
            # update the no_trips attribute of the nodes
            G_country.nodes[u_country]["no_trips"] += edge_data["no_trips"]
            G_country.nodes[v_country]["no_trips"] += edge_data["no_trips"]


In [ ]:
node_sizes = {}
for node, data in G_country.nodes(data=True):
    node_sizes[node] = data["no_trips"] * 20  #

edge_widths = []
for u, v, data in G_country.edges(data=True):
    edge_widths.append(data["no_trips"] * 0.05) 


plt.figure(figsize=(12, 10))
pos = nx.fruchterman_reingold_layout(G_country, weight='no_trips', k=4.5, seed=34) # 2, 7, 16, 17, 26, 30, 32, 24
nx.draw_networkx_edges(G_country, pos, width=edge_widths)
node_colors = [data["no_trips"] for node, data in G_country.nodes(data=True)]
sm = plt.cm.ScalarMappable(cmap='cool', norm=plt.Normalize(vmin=min(node_colors), vmax=max(node_colors)))
sm.set_array([])
nx.draw_networkx_nodes(G_country, pos, node_size=list(node_sizes.values()), node_color=node_colors, cmap='cool', alpha=0.8)
nx.draw_networkx_labels(G_country, pos, font_size=8, font_color="k", font_family="sans-serif")
cbar = plt.colorbar(sm, shrink=0.5, pad=-0.01)
cbar.ax.set_ylabel('Number of Visits', rotation=90, fontsize=15)
plt.axis("off")
plt.show()


#### Degree Centrality

In [ ]:
degree = nx.degree_centrality(G_country)

node_sizes = [degree[node]*4000 for node in G_country.nodes()]

edge_widths = []
for u, v, data in G_country.edges(data=True):
    edge_widths.append(data["no_trips"] * 0.05)  


# plot the graph
plt.figure(figsize=(12, 10))
pos = nx.fruchterman_reingold_layout(G_country, weight='no_trips', k=4.5, seed=34) # 2, 7, 16, 17, 26, 30, 32, 24
nx.draw_networkx_edges(G_country, pos, width=edge_widths)
nodes = nx.draw_networkx_nodes(G_country, pos, node_size=node_sizes, node_color=list(degree.values()), cmap='cool', alpha=0.8)
nx.draw_networkx_labels(G_country, pos, font_size=8, font_color="k", font_family="sans-serif")
sm = plt.cm.ScalarMappable(cmap='cool', norm=plt.Normalize(vmin=min(degree.values()), vmax=max(degree.values())))
sm._A = []  # this line is necessary to avoid warnings
cbar = plt.colorbar(sm, shrink=0.5, pad=-0.01)
cbar.ax.set_ylabel('Node Degree Centrality', rotation=90, fontsize=14)
plt.axis("off")
plt.show()



#### Betweenness Centrality

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

degree = nx.betweenness_centrality(G_country)


node_sizes = [degree[node]*20000 for node in G_country.nodes()]

edge_widths = []
for u, v, data in G_country.edges(data=True):
    edge_widths.append(data["no_trips"] * 0.05)  


# plot the graph
plt.figure(figsize=(12, 10))
pos = nx.fruchterman_reingold_layout(G_country, weight='no_trips', k=4.5, seed=34) # 2, 7, 16
nx.draw_networkx_edges(G_country, pos, width=edge_widths)
nodes = nx.draw_networkx_nodes(G_country, pos, node_size=node_sizes, node_color=list(degree.values()), cmap='cool', alpha=0.8)
nx.draw_networkx_labels(G_country, pos, font_size=8, font_color="k", font_family="sans-serif")
sm = plt.cm.ScalarMappable(cmap='cool', norm=plt.Normalize(vmin=min(degree.values()), vmax=max(degree.values())))
sm._A = []  # this line is necessary to avoid warnings
cbar = plt.colorbar(sm, shrink=0.5, pad=-0.01)
cbar.ax.set_ylabel('Node Betweenness Centrality', rotation=90, fontsize=14)
plt.axis("off")
plt.show()

#### Eigencentrality

In [ ]:
degree = nx.eigenvector_centrality(G_country)

node_sizes = [degree[node]*10000 for node in G_country.nodes()]

edge_widths = []
for u, v, data in G_country.edges(data=True):
    edge_widths.append(data["no_trips"] * 0.05)  


# plot the graph
plt.figure(figsize=(12, 10))
pos = nx.fruchterman_reingold_layout(G_country, weight='no_trips', k=4.5, seed=34)
nx.draw_networkx_edges(G_country, pos, width=edge_widths)
nodes = nx.draw_networkx_nodes(G_country, pos, node_size=node_sizes, node_color=list(degree.values()), cmap='cool', alpha=0.8)
nx.draw_networkx_labels(G_country, pos, font_size=8, font_color="k", font_family="sans-serif")
sm = plt.cm.ScalarMappable(cmap='cool', norm=plt.Normalize(vmin=min(degree.values()), vmax=max(degree.values())))
sm._A = [] 
cbar = plt.colorbar(sm, shrink=0.5, pad=-0.01)
cbar.ax.set_ylabel('Node Eigen Centrality', rotation=90, fontsize=14)
plt.axis("off")
plt.show()

#### Closesness Centrality

In [ ]:
degree = nx.closeness_centrality(G_country)

node_sizes = [degree[node]**4 * 10000 for node in G_country.nodes()]

edge_widths = []
for u, v, data in G_country.edges(data=True):
    edge_widths.append(data["no_trips"] * 0.05)  # scale the width based on number of trips


# plot the graph
plt.figure(figsize=(12, 10))
pos = nx.fruchterman_reingold_layout(G_country, weight='no_trips', k=4.5, seed=34)
nx.draw_networkx_edges(G_country, pos, width=edge_widths)
nodes = nx.draw_networkx_nodes(G_country, pos, node_size=node_sizes, node_color=list(degree.values()), cmap='cool', alpha=0.8)
nx.draw_networkx_labels(G_country, pos, font_size=8, font_color="k", font_family="sans-serif")
sm = plt.cm.ScalarMappable(cmap='cool', norm=plt.Normalize(vmin=min(degree.values()), vmax=max(degree.values())))
sm._A = []  
cbar = plt.colorbar(sm, shrink=0.5, pad=-0.01)
cbar.ax.set_ylabel('Node Closeness Centrality', rotation=90, fontsize=14)
plt.axis("off")
plt.show()

In [ ]:
df_G_ships_multi = nx.to_pandas_edgelist(G_ships_multi)

### Multi to Single Graph but with self loop nodes as edges

In [ ]:
G_ships = nx.Graph()

for u, v in G_ships_multi.edges():
    edge_dict = G_ships_multi[u][v]
    no_trips = len(edge_dict)
    dists = [edge_dict[i]['dist'] for i in range(no_trips)]
    time_diffs = [edge_dict[i]['time'].total_seconds() for i in range(no_trips)]

    if len(dists) > 0:
        mean_dist, std_dist = np.mean(dists), np.std(dists)
    else:
        mean_dist, std_dist = np.nan, np.nan

    if len(time_diffs) > 0:
        mean_time, std_time = datetime.timedelta(seconds=np.mean(time_diffs)), datetime.timedelta(seconds=np.std(time_diffs))
    else:
        mean_time, std_time = np.nan, np.nan

    if not G_ships.has_edge(u, v):
        G_ships.add_edge(u, v, no_trips=no_trips, mean_dist=mean_dist, std_dist=std_dist, mean_time=mean_time, std_time=std_time)
    else:
        G_ships[u][v]['no_trips'] = no_trips
        G_ships[u][v]['mean_dist'] = mean_dist
        G_ships[u][v]['std_dist'] = std_dist
        G_ships[u][v]['mean_time'] = mean_time
        G_ships[u][v]['std_time'] = std_time


# Add self-loop edges as nodes to the graph with the same attributes as the edges
for u, v, edge_attrs in G_ships.edges(data=True):
    if u == v:  # Check if it's a self-loop edge
        lat = all_nodes.loc[all_nodes['id'] == u, 'lat'].iloc[0]
        lon = all_nodes.loc[all_nodes['id'] == u, 'lon'].iloc[0]
        G_ships.add_node(u, no_trips=edge_attrs['no_trips'], mean_dist=edge_attrs['mean_dist'], std_dist=edge_attrs['std_dist'], mean_time=edge_attrs['mean_time'], std_time=edge_attrs['std_time'], lat=lat, lon=lon)


df_G_ships = nx.to_pandas_edgelist(G_ships)
df_port_nodes = df_G_ships[(df_G_ships['source'] == df_G_ships['target']) & (df_G_ships['source'].str.startswith("P"))]
df_port_nodes = df_port_nodes.drop(columns=["mean_dist", "std_dist"])
df_edges = df_G_ships[df_G_ships['source'] != df_G_ships['target']]



In [ ]:
# Count the number of edges where no_trips is 1
count_no_trips_1 = len(df_edges[df_edges['no_trips'] == 1])
percent_no_trips_1 = (count_no_trips_1 / len(df_edges)) * 100

# Print the percentage
print("Percentage of edges where no_trips is 1:", percent_no_trips_1)

In [ ]:
# Count the number of edges where no_trips is 1
count_no_trips_2 = df_edges[df_edges['no_trips'] < 10]
count_no_trips_2 = len(count_no_trips_2[count_no_trips_2['no_trips'] > 0])
percent_no_trips_2 = (count_no_trips_2 / len(df_edges)) * 100

# Print the percentage
print("Percentage of edges where no_trips is :", percent_no_trips_2)



In [ ]:
# Count the number of edges where no_trips is 1
count_no_trips_2 = df_edges[df_edges['no_trips'] > 100]
count_no_trips_2 = len(count_no_trips_2[count_no_trips_2['no_trips'] > 0])
percent_no_trips_2 = (count_no_trips_2 / len(df_edges)) * 100

# Print the percentage
print("Percentage of edges where no_trips is :", percent_no_trips_2)

#### No. Trips

In [ ]:
print(G_ships.nodes(data=True))
# Sort the edges in descending order based on the no_trips attribute
sorted_edges = sorted(G_ships.edges(data=True), key=lambda x: x[2]['no_trips'], reverse=True)

from matplotlib import colors

def G_no_trips_plot(G, undir=True, threshold=1, region="Global", plot_df=False):
    G.remove_edges_from(nx.selfloop_edges(G))
    weights = [d['no_trips'] for u, v, d in G.edges(data=True)]

    min_weight, max_weight = min(weights), max(weights)

    pos_undir = {node:(data['lon'], data['lat']) for node, data in G.nodes(data=True)}

    single_edges = [(u, v) for u, v, d in G.edges(data=True) if d['no_trips'] <= 1]
    repeated_edges = [(u, v) for u, v, d in G.edges(data=True) if d['no_trips'] > threshold]
    repeated_edges = sorted(repeated_edges, key=lambda e: G.get_edge_data(e[0], e[1])['no_trips'])

    plt, ax = crs_plot(world_bw=True, world_color=True, x_spac=20, y_spac=20)
    if plot_df:
        ax.scatter(df.lon, df.lat, c='k', edgecolor='None', marker = "x", alpha=0.7, s=0.2, linewidth=0.01)

    norm = colors.PowerNorm(vmin=min_weight, vmax=max_weight, gamma=0.7)
    cmap = plt.cm.get_cmap('gnuplot2', len(weights))
    edge_colors = [cmap(norm(G.get_edge_data(u, v)['no_trips'])) for u, v in repeated_edges]

    edge_weights = np.array([norm(G.get_edge_data(u, v)['no_trips']) for u, v in repeated_edges])

    if region == "CH":
        ax.set_extent([80, 160, -30, 50], crs=ccrs.PlateCarree())
    elif region =="EU":
        ax.set_extent([-15, 40, 25, 72], crs=ccrs.PlateCarree())
    elif region == "US":
        ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
    elif region == "ME":
        ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AF":
        ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AUS":
        ax.set_extent([100, 155, -43, 8], crs=ccrs.PlateCarree())


    nx.draw_networkx_nodes(G, pos_undir, node_color="k", node_size=0.1)
    if undir:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.1, alpha=0.3, edge_color="k", style='dashed')
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=edge_weights*8, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight)
        # plt.title("Undirected Network")

    else:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.2, edge_color="gainsboro", style='dashed', connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10) # single edges
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=0.5, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight, connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10);
        # plt.title("Directed Network")

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    # cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1, shrink=1)
    cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1, shrink=0.56)

    cbar.ax.tick_params(labelsize=8)
    cbar.ax.minorticks_on()
    cbar.set_label('Number of Trips')

# G_no_trips_plot(G_directed, undir=False, threshold=1, region="EU")
G_no_trips_plot(G_ships, undir=True, threshold=1, region="GL")

In [ ]:
import cmasher as cmr

def G_std_time_plot(G, undir=True, threshold=0, region="Global"):
    G.remove_edges_from(nx.selfloop_edges(G)) #NOTE

    weights = [(d['std_time'].total_seconds() / (24 * 60 * 60)) / (d['mean_time'].total_seconds() / (24 * 60 * 60)) for u, v, d in G.edges(data=True) if d['mean_time'] is not None]
    print(weights)
    min_weight, max_weight = np.nanmin(weights), np.nanmax(weights)
    print(min_weight, max_weight)
    pos_undir = {node:(data['lon'], data['lat']) for node, data in G.nodes(data=True)}

    # define edges
    single_edges = [(u, v) for u, v, d in G.edges(data=True) if d['no_trips'] == 1] # edges with no_trips <= 1
    repeated_edges = [(u, v) for u, v, d in G.edges(data=True) if d['mean_time'] is not None] # edges with no_trips > 1
    repeated_edges = sorted(repeated_edges, key=lambda e: G.get_edge_data(e[0], e[1])['std_time'])

    plt, ax = crs_plot(world_bw=True, world_color=True)
    norm = colors.PowerNorm(vmin=min_weight, vmax=max_weight, gamma=0.4) # fit power curve to color values
    # cmap = plt.cm.get_cmap('gnuplot2', len(weights))
    cmap = cmr.get_sub_cmap('gnuplot2', 0, 0.95)


    edge_weights = np.array([norm(G.get_edge_data(u, v)['std_dist']) for u, v in repeated_edges])

    # edge_colors = [cmap(norm(G.get_edge_data(u, v)['std_time'].total_seconds() / (24 * 60 * 60) )) for u, v in repeated_edges]
    edge_colors = [cmap(norm((G.get_edge_data(u, v)['std_time'].total_seconds() / (24 * 60 * 60) )/ (G.get_edge_data(u, v)['mean_time'].total_seconds() / (24 * 60 * 60)))) for u, v in repeated_edges]
    
    if region == "CH":
        ax.set_extent([90, 150, -30, 50], crs=ccrs.PlateCarree())
    elif region =="EU":
        ax.set_extent([-20, 50, 20, 75], crs=ccrs.PlateCarree())
    elif region == "US":
        ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
    elif region == "ME":
        ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AF":
        ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())


    # nx.draw_networkx_nodes(G, pos_undir, node_color="k", node_size=0.1)
    if undir:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.2, edge_color="gainsboro", style='dashed') # single edges
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=edge_weights*0.08, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight) # repeated edges

    else:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.2, edge_color="gainsboro", style='dashed', connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10) # single edges
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=0.5, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight, connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10);

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1)
    cbar.ax.tick_params(labelsize=8) 
    cbar.ax.minorticks_on()
    cbar.set_label('Time Coefficient of Variation')

G_std_time_plot(G_ships, undir=True, threshold=1, region="World")

In [ ]:

def G_std_time_plot(G, undir=True, threshold=0, region="Global"):
    G.remove_edges_from(nx.selfloop_edges(G)) #NOTE

    # weights = [d['std_time'].total_seconds() / (24 * 60 * 60) for u, v, d in G.edges(data=True)]
    weights = [(d['std_dist']) / (d['mean_dist']) for u, v, d in G.edges(data=True) if d['mean_time'] is not None]
    print(weights)
    min_weight, max_weight = np.nanmin(weights), np.nanmax(weights)
    print(min_weight, max_weight)
    # get longitude and latitude positions of nodes
    pos_undir = {node:(data['lon'], data['lat']) for node, data in G.nodes(data=True)}

    # define edges
    single_edges = [(u, v) for u, v, d in G.edges(data=True) if pd.isnull(d['std_dist'])] # edges with no_trips <= 1

    # repeated_edges = [(u, v) for u, v, d in G.edges(data=True) if d['std_time'].total_seconds() / (24 * 60 * 60) > threshold] # edges with no_trips > 1
    repeated_edges = [(u, v) for u, v, d in G.edges(data=True) if (d['std_dist']) > threshold and d['mean_dist'] is not None] # edges with no_trips > 1
    repeated_edges = sorted(repeated_edges, key=lambda e: G.get_edge_data(e[0], e[1])['std_dist'])

    plt, ax = crs_plot(world_bw=True, world_color=True)
    norm = colors.PowerNorm(vmin=min_weight, vmax=max_weight, gamma=0.6) # fit power curve to color values
    cmap = plt.cm.get_cmap('gnuplot2', len(weights))

    edge_colors = [cmap(norm(G.get_edge_data(u, v)['std_dist'])) for u, v in repeated_edges]

    edge_weights = np.array([norm(G.get_edge_data(u, v)['std_dist']) for u, v in repeated_edges])
    print(edge_weights)


    # edge_colors = [cmap(norm(G.get_edge_data(u, v)['std_time'].total_seconds() / (24 * 60 * 60) )) for u, v in repeated_edges]
    edge_colors = [cmap(norm((G.get_edge_data(u, v)['std_dist'])/ (G.get_edge_data(u, v)['mean_dist']))) for u, v in repeated_edges]
    
    if region == "CH":
        ax.set_extent([90, 150, -30, 50], crs=ccrs.PlateCarree())
    elif region =="EU":
        ax.set_extent([-20, 50, 20, 75], crs=ccrs.PlateCarree())
    elif region == "US":
        ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
    elif region == "ME":
        ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AF":
        ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())


    # nx.draw_networkx_nodes(G, pos_undir, node_color="k", node_size=0.1)
    if undir:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.2, edge_color="gainsboro", style='dashed') # single edges
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=edge_weights*0.02, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight) # repeated edges

    else:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.2, edge_color="gainsboro", style='dashed', connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10) # single edges
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=0.5, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight, connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10);

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1)
    cbar.ax.tick_params(labelsize=8) 
    cbar.ax.minorticks_on()
    cbar.set_label('Time Standard Deviation (days)')
    cbar.set_label('Distance Coefficient of Variation')

# G_no_trips_plot(G_directed, undir=False, threshold=1, region="EU")
G_std_time_plot(G_ships, undir=True, threshold=1, region="World")

### Different Voyages between Same Ports

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.cm import ScalarMappable
%matplotlib inline
node_comb = (224.0, 18.0)

multi_edges = {}
for u, v, attr in G_ships_multi.edges(data=True):
    if attr.get('port_comb') == node_comb and u != v:
        key = (u, v)
        if key not in multi_edges:
            multi_edges[key] = 0
        multi_edges[key] += 1

cmap = cm.get_cmap('gnuplot2_r')
edge_counts = list(multi_edges.values())
edge_norm = colors.Normalize(vmin=min(edge_counts), vmax=max(edge_counts))

plt, ax = crs_plot(world_bw=True, world_color=True, x_spac=10, y_spac=10)
edge_list = [(u, v) for (u, v) in G_ships_multi.edges() if u != v]

nx.draw_networkx_nodes(G_ships_multi, pos_ships, alpha=0.5, node_color="k", node_size=0.1).set_zorder(1)
nx.draw_networkx_edges(G_ships_multi, pos_ships, width=0.1, alpha=0.1, edge_color="k", edgelist=edge_list).set_zorder(1)

sorted_edges = sorted(multi_edges.items(), key=lambda x: x[1], reverse=False)

pos_ships = {node:(data['lon'], data['lat']) for node, data in G_ships_multi.nodes(data=True)}

for edge, count in sorted_edges:
    color = cmap(edge_norm(count))
    nx.draw_networkx_edges(G_ships_multi, pos_ships, edgelist=[edge], width=2, edge_color=color).set_zorder(2)

edges = [(u, v) for u, v, attr in G_ships_multi.edges(data=True) 
         if attr.get('port_comb') == node_comb and u != v]
nodes_red = set(u for u, v in edges) | set(v for u, v in edges)
nodes_blue = {node for node in nodes_red if G_ships_multi.nodes[node]['node_type'] == "P"}

nx.draw_networkx_nodes(G_ships_multi, pos_ships, nodelist=nodes_red - nodes_blue, node_size=0.5, node_color='k').set_zorder(3)
nx.draw_networkx_nodes(G_ships_multi, pos_ships, nodelist=["P" + str(int(node_comb[0]))], node_size=20, node_color='limegreen', node_shape="d", label='Start Port').set_zorder(4)
nx.draw_networkx_nodes(G_ships_multi, pos_ships, nodelist=["P" + str(int(node_comb[1]))], node_size=20, node_color='r', node_shape="d", label='End Port').set_zorder(4)
# plt.legend(frameon=False, loc="upper right")
sm = ScalarMappable(cmap=cmap, norm=edge_norm)
sm.set_array([])
# cbar = plt.colorbar(sm, orientation='vertical', shrink=1, aspect=40, pad=0.1)
cbar = plt.colorbar(sm, orientation='horizontal', shrink=1, aspect=40, pad=0.06)

# cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.85, aspect=40, ticks=np.arange(1, 5+1, 1), pad=0.06)
cbar.ax.tick_params(labelsize=8) 
cbar.set_label('No. Trips')

total_imos = set()
for u, v, attr in G_ships_multi.edges(data=True):
    if attr.get('port_comb') == node_comb:
        total_imos.add(attr.get('ship_imo'))

print(f"Total number of IMOs for port combination {node_comb}: {len(total_imos)}")

edge_coords = [(pos_ships[start], pos_ships[end]) for start, end in multi_edges.keys()]
x_coords = [coord[0][0] for coord in edge_coords] + [coord[1][0] for coord in edge_coords]
y_coords = [coord[0][1] for coord in edge_coords] + [coord[1][1] for coord in edge_coords]
plt.xlim(min(x_coords)-1, max(x_coords)+1)
plt.ylim(min(y_coords)-1, max(y_coords)+1)
plt.show()

In [ ]:
#(224, 18)
#(23, 224)
#(34, 301)
#(139, 18)
df_filtered = df_G_ships_multi.loc[(df_G_ships_multi['port_comb'] == (139, 18))]
rows = []
for name, group in df_filtered.groupby('ship_imo'):
    time_sum = group.time.sum()
    dist_sum = group.dist.sum()
    rows.append({'imo': name, 'data': group, 'time_sum': time_sum, 'dist_sum': dist_sum})
df_grouped = pd.DataFrame(rows)
print(df_grouped.time_sum.std().total_seconds()/(24*60*60))
print(df_grouped.time_sum.std()/df_grouped.time_sum.mean())
print("-------")
print(df_grouped.dist_sum.std())
print(df_grouped.dist_sum.mean())

print(df_grouped.dist_sum.std()/df_grouped.dist_sum.mean())

In [ ]:

pos_ships = {node:(data['lon'], data['lat']) for node, data in G_ships_multi.nodes(data=True)}


plt, ax = crs_plot(world_bw=True, world_color=False)

node_labels = {n: n if n in ['P12', 'P13'] else '' for n in G_ships_multi.nodes()}
nx.draw_networkx_nodes(G_ships_multi, pos_ships, node_color="k", node_size=0.5);
# nx.draw_networkx_labels(G_ships_multi, pos_ships, labels=node_labels, font_size=4);
nx.draw_networkx_labels(G_ships_multi, pos_ships, font_size=4);


In [ ]:
import matplotlib.pyplot as plt

port_comb_list = []
total_imos_list = []

for node_comb in unique_port_combs:
    if node_comb[0] != node_comb[1]:
        # get total_imos for the current port_combination
        total_imos = set()
        for u, v, attr in G_ships_multi.edges(data=True):
            if attr.get('port_comb') == node_comb:
                total_imos.add(attr.get('ship_imo'))
        # append to lists
        port_comb_list.append(node_comb)
        total_imos_list.append(len(total_imos))

plt.scatter(port_comb_list, total_imos_list)
plt.xlabel('Port Combination')
plt.ylabel('Total IMOs')
plt.title('Zipf\'s Law Plot')
plt.show()

## Directed Network

In [ ]:
G_ships_multi_dir = nx.MultiDiGraph()

for imo, row in df_ship_network_T.iterrows():
    row_data = row.T.iloc[0]

    for _, sub_row in row_data.iterrows():
        port_comb = sub_row[0]
        for _, node_group in sub_row[-1].iterrows():
            node_comb = node_group['betw_node']
            G_ships_multi_dir.add_edge(node_comb[0], node_comb[1], 
                                       time=node_group.loc['time_diff'], 
                                       dist=node_group.loc['dist'], 
                                       start_month=node_group.loc['start_month'],
                                       ship_imo=imo, 
                                       port_comb=port_comb)

            
for i, row in all_nodes.iterrows():
    if row.id.startswith("N"):
        node_type = "N"
    elif row.id.startswith("P"):
        node_type = "P"
    G_ships_multi_dir.add_node(row.id, lat=row['lat'], lon=row['lon'], node_type=node_type)




In [ ]:
import numpy as np
import datetime

# Create an empty directed graph
G_ships_dir = nx.DiGraph()

for u, v, edge_dict in G_ships_multi_dir.edges(keys=True):
    no_trips = len(G_ships_multi_dir.get_edge_data(u, v))
    dists = [G_ships_multi_dir[u][v][key]['dist'] for key in G_ships_multi_dir[u][v]]
    time_diffs = [G_ships_multi_dir[u][v][key]['time'].total_seconds() for key in G_ships_multi_dir[u][v]]

    if len(dists) > 0:
        mean_dist, std_dist = np.mean(dists), np.std(dists)
    else:
        mean_dist, std_dist = np.nan, np.nan

    if len(time_diffs) > 0:
        mean_time, std_time = datetime.timedelta(seconds=np.mean(time_diffs)), datetime.timedelta(seconds=np.std(time_diffs))
    else:
        mean_time, std_time = np.nan, np.nan

    G_ships_dir.add_edge(u, v, no_trips=no_trips, mean_dist=mean_dist, std_dist=std_dist, mean_time=mean_time, std_time=std_time)

# # Add nodes to the graph
# for i, row in all_nodes.iterrows():
#     if row.id.startswith("N"):
#         node_type = "N"
#         G_ships_dir.add_node(row.id, lat=row['lat'], lon=row['lon'], node_type=node_type)

# Add self-loop edges as nodes to the graph with the same attributes as the edges
for u, v, edge_attrs in G_ships_dir.edges(data=True):
    if u == v:  # Check if it's a self-loop edge
        lat = all_nodes.loc[all_nodes['id'] == u, 'lat'].iloc[0]
        lon = all_nodes.loc[all_nodes['id'] == u, 'lon'].iloc[0]

        G_ships_dir.add_node(u, no_trips=edge_attrs['no_trips'], mean_dist=edge_attrs['mean_dist'], std_dist=edge_attrs['std_dist'], mean_time=edge_attrs['mean_time'], std_time=edge_attrs['std_time'], lat=lat, lon=lon)


In [ ]:
G_ships_dir.remove_edges_from(nx.selfloop_edges(G_ships_dir))
%matplotlib inline


# Sort the edges in descending order based on the no_trips attribute
# sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]['no_trips'], reverse=True)

# Print the sorted edges and attributes of the new graph
# for u, v, data in sorted_edges:
#     print(f"({u}, {v}): {data['no_trips']}")

from matplotlib import colors

def G_no_trips_plot_2(G, undir=True, threshold=1, region="Global", plot_df=False):
    G.remove_edges_from(nx.selfloop_edges(G))
    weights = [d['no_trips'] for u, v, d in G.edges(data=True)]
    min_weight, max_weight = min(weights), max(weights)

    # get longitude and latitude positions of nodes

    print(G.nodes(data=True))
    pos_undir = {node:(data['lon'], data['lat']) for node, data in G.nodes(data=True)}

    # define edges
    single_edges = [(u, v) for u, v, d in G.edges(data=True) if d['no_trips'] <= 1] # edges with no_trips <= 1

    repeated_edges = [(u, v) for u, v, d in G.edges(data=True) if d['no_trips'] > threshold] # edges with no_trips > 1
    repeated_edges = sorted(repeated_edges, key=lambda e: G.get_edge_data(e[0], e[1])['no_trips'],  reverse=False)

    plt, ax = crs_plot(world_bw=True, world_color=True, x_spac=2, y_spac=2)

    if plot_df:
        ax.scatter(df.lon, df.lat, c='k', edgecolor='None', marker = "x", alpha=0.7, s=0.2, linewidth=0.01)

    norm = colors.PowerNorm(vmin=min_weight, vmax=max_weight, gamma=0.6) # fit power curve to color values
    cmap = plt.cm.get_cmap('gnuplot2', len(weights))
    edge_colors = [cmap(norm(G.get_edge_data(u, v)['no_trips'])) for u, v in repeated_edges]

    edge_weights = np.array([norm(G.get_edge_data(u, v)['no_trips']) for u, v in repeated_edges])
    if region == "CH":
        ax.set_extent([80, 160, -30, 50], crs=ccrs.PlateCarree())
    elif region =="EU":
        ax.set_extent([-20, 44, 20, 72], crs=ccrs.PlateCarree())
    elif region == "US":
        ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
    elif region == "ME":
        ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AF":
        ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AUS":
        ax.set_extent([100, 155, -43, 8], crs=ccrs.PlateCarree())
    elif region == "SING":
        ax.set_extent([100, 155, -22, 20], crs=ccrs.PlateCarree())
    elif region == "MECH":
        ax.set_extent([40, 140, -30, 20], crs=ccrs.PlateCarree())
    elif region =="DK":
        ax.set_extent([8, 16, 53.5, 58], crs=ccrs.PlateCarree())

    ports = [node for node in G.nodes() if node.startswith("P")]    
    nodes = [node for node in G.nodes() if node.startswith("N")]

    nx.draw_networkx_nodes(G, pos_undir, nodelist=nodes, node_color="k", node_size=10)
    nx.draw_networkx_nodes(G, pos_undir, nodelist=ports,  node_color="red", node_shape="^", node_size=15)

    if undir:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.005, edge_color="k", style='dashed') # single edges gainsboro
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=edge_weights*0.2, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight) # repeated edges

    else:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.05, edge_color="k", style='dashed', connectionstyle="arc3,rad=0.1", arrowstyle='->', arrowsize=5, node_size=2) # single edges
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=edge_weights*5,edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight, connectionstyle="arc3,rad=0.11", arrowstyle='->', arrowsize=5, node_size=2);

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1, shrink=1)
    cbar.ax.tick_params(labelsize=8)
    cbar.ax.minorticks_on()
    cbar.set_label('Number of Trips')

G_no_trips_plot_2(G_ships_dir, undir=False, threshold=1, region="DK")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# net number of trips between each pair of nodes
net_trips = {}
G_ships_dir.remove_edges_from(nx.selfloop_edges(G_ships_dir))
for u, v, data in G_ships_dir.edges(data=True):
    u_v_trips = data.get('no_trips', 0)
    v_u_trips = G_ships_dir[v].get(u, {}).get('no_trips', 0)
    net_trips[(u, v)] = abs(u_v_trips - v_u_trips)  # compute absolute difference

# Plot the graph with edges colored by net number of trips
plt, ax = crs_plot(world_bw=True, x_spac=45, y_spac=45)
pos = {node: (G_ships_dir.nodes[node]['lon'], G_ships_dir.nodes[node]['lat']) for node in G_ships_dir.nodes()}


repeated_edges = [(u, v) for u, v, d in G_ships_dir.edges(data=True) if d['no_trips'] > 20] # edges with no_trips > 1
repeated_edges = sorted(repeated_edges, key=lambda e: G_ships_dir.get_edge_data(e[0], e[1])['no_trips'])

edge_weights = np.array([norm(G_ships_dir.get_edge_data(u, v)['no_trips']) for u, v in repeated_edges])

mappable = cm.ScalarMappable(norm=plt.Normalize(vmin=edge_weights.min(), vmax=edge_weights.max()), cmap=plt.cm.magma)

# Draw edges with arrows in one direction only
visited_edges = set()
for u, v, data in G_ships_dir.edges(data=True):
    if (u, v) in visited_edges or (v, u) in visited_edges:
        continue
    visited_edges.add((u, v))
    visited_edges.add((v, u))

    if G_ships_dir[v].get(u, {}).get('no_trips', 0) > data.get('no_trips', 0):
        arrowstyle = '<-'
    else:
        arrowstyle = '->'
    edge_color = mappable.to_rgba(data['no_trips'])
    edge_width = abs(net_trips[(u, v)]) / 25
    nx.draw_networkx_edges(G_ships_dir, pos, edgelist=[(u, v)], width=edge_width, edge_color=edge_color, arrowstyle=arrowstyle, connectionstyle="arc3,rad=0.08", arrowsize=5, node_size=1)

region = "GLOBAL"
# Set the extent of the plot based on the region
if region == "CH":
    ax.set_extent([90, 150, -30, 50], crs=ccrs.PlateCarree())
elif region =="EU":
    ax.set_extent([-20, 50, 20, 75], crs=ccrs.PlateCarree())
elif region == "US":
    ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
elif region == "ME":
    ax.set_extent([20, 110, -20, 40], crs=ccrs.PlateCarree())
elif region == "MEL":
    ax.set_extent([20, 110, -50, 20], crs=ccrs.PlateCarree())
elif region == "AF":
    ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())
elif region =="DK":
    ax.set_extent([-5, 30, 46, 64], crs=ccrs.PlateCarree())

cbar = plt.colorbar(mappable, aspect=50, orientation="horizontal", pad=0.1)
cbar.ax.tick_params(labelsize=8) 
cbar.ax.minorticks_on()
cbar.set_label('Net No. Trips')
plt.show()

In [ ]:
%matplotlib inline
net_trips = {}
G_ships_dir.remove_edges_from(nx.selfloop_edges(G_ships_dir))

for u, v, data in G_ships_dir.edges(data=True):
    u_v_trips = data.get('no_trips', 0)
    v_u_trips = G_ships_dir[v].get(u, {}).get('no_trips', 0)
    net_trips[(u, v)] = abs(u_v_trips - v_u_trips) 

# sort edges by net_trips value in descending order
edges_sorted = sorted(G_ships_dir.edges(data=True), key=lambda x: net_trips[(x[0], x[1])], reverse=False)

visited_edges = set()
plt, ax = crs_plot(world_bw=True, world_color=True, x_spac=45, y_spac=45)

edge_weights = [data['no_trips'] for u, v, data in G_ships_dir.edges(data=True)]
norm = colors.PowerNorm(gamma=0.4, vmin=min(edge_weights), vmax=max(edge_weights))
mappable = cm.ScalarMappable(norm=norm, cmap=plt.cm.gnuplot2)

# loop over sorted edges and draw them in order
for u, v, data in edges_sorted:
    if (u, v) in visited_edges or (v, u) in visited_edges:
        continue
    visited_edges.add((u, v))
    visited_edges.add((v, u))

    if net_trips[(u, v)] > 2: 
        if G_ships_dir[v].get(u, {}).get('no_trips', 0) > data.get('no_trips', 0):
            arrowstyle = '<-'
        else:
            arrowstyle = '->'

        if net_trips[(u, v)] > 10:
            edge_width = 2.5
        else:
            edge_width = norm(net_trips[(u, v)])*5
        edge_color = mappable.to_rgba(data['no_trips'])  # apply the norm to the edge color

        nx.draw_networkx_edges(G_ships_dir, pos, edgelist=[(u, v)], width=edge_width, edge_color=edge_color, arrowstyle=arrowstyle, connectionstyle="arc3,rad=0.08", arrowsize=4, node_size=1)

region = "CUST"

if region == "CH":
    ax.set_extent([90, 150, -30, 50], crs=ccrs.PlateCarree())
elif region =="EU":
    ax.set_extent([-20, 50, 20, 75], crs=ccrs.PlateCarree())
elif region == "US":
    ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
elif region == "ME":
    ax.set_extent([20, 110, -20, 40], crs=ccrs.PlateCarree())
elif region == "MEL":
    ax.set_extent([20, 110, -50, 20], crs=ccrs.PlateCarree())
elif region == "AF":
    ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())
elif region =="DK":
    ax.set_extent([-5, 30, 46, 64], crs=ccrs.PlateCarree())
elif region =="CUST":
    ax.set_extent([-100, 180, -55, 80], crs=ccrs.PlateCarree())
    
cbar = plt.colorbar(mappable, aspect=50, orientation="horizontal", pad=0.1)
cbar.ax.tick_params(labelsize=8) 
cbar.ax.minorticks_on()
cbar.set_label('Net No. Trips')
plt.show()

In [ ]:
net_trips = {}
G_ships_dir.remove_edges_from(nx.selfloop_edges(G_ships_dir))

# compute net_trips dictionary
for u, v, data in G_ships_dir.edges(data=True):
    u_v_trips = data.get('no_trips', 0)
    v_u_trips = G_ships_dir[v].get(u, {}).get('no_trips', 0)
    net_trips[(u, v)] = abs(u_v_trips - v_u_trips)  # compute absolute difference

# sort edges by net_trips value in descending order
edges_sorted = sorted(G_ships_dir.edges(data=True), key=lambda x: net_trips[(x[0], x[1])], reverse=False)

visited_edges = set()
plt, ax = crs_plot(world_bw=True, world_color=True, x_spac=10, y_spac=10)

edge_weights = [data['no_trips'] for u, v, data in G_ships_dir.edges(data=True)]
norm = colors.PowerNorm(gamma=0.4, vmin=min(edge_weights), vmax=max(edge_weights))
mappable = cm.ScalarMappable(norm=norm, cmap=plt.cm.gnuplot2)

# loop over sorted edges and draw them in order
for u, v, data in edges_sorted:
    if (u, v) in visited_edges or (v, u) in visited_edges:
        continue
    visited_edges.add((u, v))
    visited_edges.add((v, u))

    if net_trips[(u, v)] > 2: 
        if G_ships_dir[v].get(u, {}).get('no_trips', 0) > data.get('no_trips', 0):
            arrowstyle = '<-'
        else:
            arrowstyle = '->'

        if net_trips[(u, v)] > 10:
            edge_width = 3
        else:
            edge_width = norm(net_trips[(u, v)])*8
        edge_color = mappable.to_rgba(data['no_trips'])  # apply the norm to the edge color

        nx.draw_networkx_edges(G_ships_dir, pos, edgelist=[(u, v)], width=edge_width, edge_color=edge_color, arrowstyle=arrowstyle, connectionstyle="arc3,rad=0.08", arrowsize=5, node_size=1)

region = "CUST"
if region == "CH":
    ax.set_extent([90, 150, -30, 50], crs=ccrs.PlateCarree())
elif region =="EU":
    ax.set_extent([-20, 50, 20, 75], crs=ccrs.PlateCarree())
elif region == "US":
    ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
elif region == "ME":
    ax.set_extent([20, 110, -20, 40], crs=ccrs.PlateCarree())
elif region == "MEL":
    ax.set_extent([20, 110, -50, 20], crs=ccrs.PlateCarree())
elif region == "AF":
    ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())
elif region =="DK":
    ax.set_extent([-5, 30, 46, 64], crs=ccrs.PlateCarree())
elif region =="CUST":
    ax.set_extent([-12, 33, 30, 62], crs=ccrs.PlateCarree())
    


### Seasonal Variation

In [ ]:
import networkx as nx
import pandas as pd

import pycountry_convert as pc

def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name


def create_G_seasons(months_Qs):
    G_seasonal_multi = nx.MultiDiGraph()
    for u, v, edge_dict in G_ships_multi.edges(data=True):
        if 'start_month' in edge_dict and edge_dict['start_month'] in months_Qs:
            G_seasonal_multi.add_edge(u, v, **edge_dict)

    G_seasonal = nx.Graph()

    for u, v in G_seasonal_multi.edges():
        edge_dict = G_seasonal_multi[u][v]
        no_trips = len(edge_dict)
        dists = [edge_dict[i]['dist'] for i in range(no_trips)]
        time_diffs = [edge_dict[i]['time'].total_seconds() for i in range(no_trips)]

        if len(dists) > 0:
            mean_dist, std_dist = np.mean(dists), np.std(dists)
        else:
            mean_dist, std_dist = np.nan, np.nan

        if len(time_diffs) > 0:
            mean_time, std_time = datetime.timedelta(seconds=np.mean(time_diffs)), datetime.timedelta(seconds=np.std(time_diffs))
        else:
            mean_time, std_time = np.nan, np.nan

        if not G_seasonal.has_edge(u, v):
            G_seasonal.add_edge(u, v, no_trips=no_trips, mean_dist=mean_dist, std_dist=std_dist, mean_time=mean_time, std_time=std_time)
        else:
            G_seasonal[u][v]['no_trips'] = no_trips
            G_seasonal[u][v]['mean_dist'] = mean_dist
            G_seasonal[u][v]['std_dist'] = std_dist
            G_seasonal[u][v]['mean_time'] = mean_time
            G_seasonal[u][v]['std_time'] = std_time


    # Add self-loop edges as nodes to the graph with the same attributes as the edges
    for u, v, edge_attrs in G_seasonal.edges(data=True):
        if u == v:  # Check if it's a self-loop edge
            lat = all_nodes.loc[all_nodes['id'] == u, 'lat'].iloc[0]
            lon = all_nodes.loc[all_nodes['id'] == u, 'lon'].iloc[0]
            country = all_nodes.loc[all_nodes['id'] == u, 'country'].iloc[0]
            # continent = country_to_continent(country)
            G_seasonal.add_node(u, no_trips=edge_attrs['no_trips'], mean_dist=edge_attrs['mean_dist'], std_dist=edge_attrs['std_dist'], mean_time=edge_attrs['mean_time'], std_time=edge_attrs['std_time'], lat=lat, lon=lon, country=country)

    return G_seasonal

months_Q1 = [1,2,3]
months_Q2 = [4,5,6]
months_Q3 = [7,8,9]
months_Q4 = [10,11,12]

G_Q1 = create_G_seasons(months_Q1)
G_Q2 = create_G_seasons(months_Q2)
G_Q3 = create_G_seasons(months_Q3)
G_Q4 = create_G_seasons(months_Q4)

df_G_Q1 = nx.to_pandas_edgelist(G_Q1)
df_G_Q2 = nx.to_pandas_edgelist(G_Q2)
df_G_Q3 = nx.to_pandas_edgelist(G_Q3)
df_G_Q4 = nx.to_pandas_edgelist(G_Q4)



In [ ]:
from matplotlib import colors

def G_no_trips_plot(G, undir=True, threshold=1, region="Global", plot_df=False):
    sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]['no_trips'], reverse=True)


    # nodes with no data
    empty_nodes = [node for node, data in G.nodes(data=True) if not data]
    G.remove_nodes_from(empty_nodes)


    G.remove_edges_from(nx.selfloop_edges(G)) 
    weights = [d['no_trips'] for u, v, d in G.edges(data=True)]
    min_weight, max_weight = min(weights), max(weights)

    # get longitude and latitude positions of nodes

    print(G.nodes(data=True))
    pos_undir = {node:(data['lon'], data['lat']) for node, data in G.nodes(data=True)}

    # define edges
    single_edges = [(u, v) for u, v, d in G.edges(data=True) if d['no_trips'] <= 1] # edges with no_trips <= 1

    repeated_edges = [(u, v) for u, v, d in G.edges(data=True) if d['no_trips'] > threshold] # edges with no_trips > 1
    repeated_edges = sorted(repeated_edges, key=lambda e: G.get_edge_data(e[0], e[1])['no_trips'])

    plt, ax = crs_plot(world_bw=True, world_color=False)

    if plot_df:
        ax.scatter(df.lon, df.lat, c='k', edgecolor='None', marker = "x", alpha=0.7, s=0.2, linewidth=0.01)

    norm = colors.PowerNorm(vmin=min_weight, vmax=max_weight, gamma=0.5) # fit power curve to color values
    cmap = plt.cm.get_cmap('magma_r', len(weights))
    edge_colors = [cmap(norm(G.get_edge_data(u, v)['no_trips'])) for u, v in repeated_edges]

    if region == "CH":
        ax.set_extent([80, 160, -30, 50], crs=ccrs.PlateCarree())
    elif region =="EU":
        ax.set_extent([-20, 44, 20, 72], crs=ccrs.PlateCarree())
    elif region == "US":
        ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
    elif region == "ME":
        ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AF":
        ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())
    elif region == "AUS":
        ax.set_extent([100, 155, -43, 8], crs=ccrs.PlateCarree())


    nx.draw_networkx_nodes(G, pos_undir, node_color="k", node_size=0.1)
    if undir:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.2, edge_color="gainsboro", style='dashed') # single edges gainsboro
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=0.5, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight) # repeated edges

    else:
        nx.draw_networkx_edges(G, pos_undir, edgelist=single_edges, width=0.2, edge_color="gainsboro", style='dashed', connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10) # single edges
        nx.draw_networkx_edges(G, pos_undir, edgelist=repeated_edges, width=0.5, edge_color=edge_colors, edge_cmap=cmap, style='solid', edge_vmin=min_weight, edge_vmax=max_weight, connectionstyle="arc3,rad=0.08", arrowstyle='->', arrowsize=2, node_size=10);

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    # cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1)
    cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1, shrink=0.5)
    cbar.ax.tick_params(labelsize=8)
    cbar.ax.minorticks_on()
    cbar.set_label('Number of Trips')

G_no_trips_plot(G_Q1, undir=True, threshold=1, region="global")
G_no_trips_plot(G_Q2, undir=True, threshold=1, region="global")
G_no_trips_plot(G_Q3, undir=True, threshold=1, region="global")
G_no_trips_plot(G_Q4, undir=True, threshold=1, region="global")


In [ ]:
import matplotlib.patches as mpatches

self_loops_Q1 = {edge for edge in G_Q1.edges() if edge[0] == edge[1] and edge[0].startswith('P')}
self_loops_Q2 = {edge for edge in G_Q2.edges() if edge[0] == edge[1] and edge[0].startswith('P')}
self_loops_Q3 = {edge for edge in G_Q3.edges() if edge[0] == edge[1] and edge[0].startswith('P')}
self_loops_Q4 = {edge for edge in G_Q4.edges() if edge[0] == edge[1] and edge[0].startswith('P')}
self_loops_all = self_loops_Q1 & self_loops_Q2 & self_loops_Q3 & self_loops_Q4

mean_times_all_Q = [[G_Q1[edge[0]][edge[1]]['mean_time'].total_seconds() / (60*60*24) if edge in self_loops_Q1 else 0,
                     G_Q2[edge[0]][edge[1]]['mean_time'].total_seconds() / (60*60*24) if edge in self_loops_Q2 else 0,
                     G_Q3[edge[0]][edge[1]]['mean_time'].total_seconds() / (60*60*24) if edge in self_loops_Q3 else 0,
                     G_Q4[edge[0]][edge[1]]['mean_time'].total_seconds() / (60*60*24) if edge in self_loops_Q4 else 0,
                     edge[0]] for edge in self_loops_all]


mean_times_all_Q = pd.DataFrame(mean_times_all_Q)
mean_times_all_Q["COV"] = mean_times_all_Q.std(axis=1)/(mean_times_all_Q.mean(axis=1))
mean_times_all_Q['country'] = mean_times_all_Q.apply(lambda row: all_nodes.loc[all_nodes["id"] == row[4], 'country'].iloc[0], axis=1)

mean_times_china_Q = mean_times_all_Q.loc[mean_times_all_Q['country'] == 'China']

common_self_loops_P_china = mean_times_china_Q[4].tolist()

mean_times_Q1_china = mean_times_china_Q[0].tolist()
mean_times_Q2_china = mean_times_china_Q[1].tolist()
mean_times_Q3_china = mean_times_china_Q[2].tolist()
mean_times_Q4_china = mean_times_china_Q[3].tolist()

x = range(len(common_self_loops_P_china))
colors = ['#FF5C74', '#E3ABAB', '#255C99', '#7EA3CC']
labels = ['Q1', 'Q2', 'Q3', 'Q4']

legend_patches = [mpatches.Patch(color=colors[i], label=labels[i]) for i in range(len(labels))]

width = 0.2
plt.figure(figsize=(9,3))
plt.bar([i-width*3/2 for i in x], mean_times_Q1_china, width=width, color=colors[0], label='Q1')
plt.bar([i-width/2 for i in x], mean_times_Q2_china, width=width, color=colors[1], label='Q2')
plt.bar([i+width/2 for i in x], mean_times_Q3_china, width=width, color=colors[2], label='Q3')
plt.bar([i+width*3/2 for i in x], mean_times_Q4_china, width=width, color=colors[3], label='Q4')
plt.xticks(x, common_self_loops_P_china, rotation=90)
plt.ylabel('Mean time (days)')
plt.xlabel('Ports in China')
plt.legend(handles=legend_patches)
plt.show()


### NODE CENTRALITY

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
import cmasher as cmr

G_ships.remove_edges_from(nx.selfloop_edges(G_ships))

# create a new edge attribute containing the converted values
for u, v, data in G_ships.edges(data=True):
    data['mean_time_seconds'] = data['mean_time'].total_seconds()

# degree = nx.betweenness_centrality(G_ships, weight='mean_dist')
# degree = nx.eigenvector_centrality(G_ships, weight='no_trips', max_iter=300)
degree = nx.betweenness_centrality(G_ships, weight='mean_time_seconds')


node_sizes = [degree[node]*50 for node in G_ships.nodes()]
print(node_sizes)

edge_widths = []
for u, v, data in G_ships.edges(data=True):
    edge_widths.append(data["no_trips"] * 0.01)  # scale the width based on number of trips


cmap = cmr.get_sub_cmap('gnuplot2', 0, 0.95)
print(G_ships.nodes(data=True))

plt, ax = crs_plot(world_bw=True, world_color=True)
pos = {node:(data['lon'], data['lat']) for node, data in G_ships.nodes(data=True)}
nx.draw_networkx_edges(G_ships, pos, width=edge_widths)
nodes = nx.draw_networkx_nodes(G_ships, pos, node_size=node_sizes, node_color=list(degree.values()), cmap=cmap, alpha=0.8)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min(degree.values()), vmax=max(degree.values())))
sm._A = []  
cbar = plt.colorbar(sm, aspect=50, orientation="horizontal", pad=0.1, shrink=1)
cbar.ax.tick_params(labelsize=8)
cbar.ax.minorticks_on()
cbar.set_label('Node Betweenness Centality')

region="GL"

if region == "CH":
    ax.set_extent([80, 160, -30, 50], crs=ccrs.PlateCarree())
elif region =="EU":
    ax.set_extent([-20, 44, 20, 72], crs=ccrs.PlateCarree())
elif region == "US":
    ax.set_extent([-170, -45, 0, 70], crs=ccrs.PlateCarree())
elif region == "ME":
    ax.set_extent([20, 100, -45, 40], crs=ccrs.PlateCarree())
elif region == "AF":
    ax.set_extent([-20, 60, -45, 40], crs=ccrs.PlateCarree())
elif region == "AUS":
    ax.set_extent([100, 155, -43, 8], crs=ccrs.PlateCarree())

plt.show()